In [2]:
import pandas as pd
from pandas import DataFrame, Series
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import datetime
import time
import json
import re
import psycopg2
from pymongo import MongoClient
from email.mime.multipart import MIMEMultipart
from email.mime.application import MIMEApplication
from email.mime.text import MIMEText
from smtplib import SMTP_SSL
from pathlib import Path
import xlsxwriter
from io import BytesIO
import dateutil
from dateutil.relativedelta import relativedelta, MO, SU
import os

In [3]:
client = MongoClient(host='10.10.224.28',
                     port=27017,
                     username='vida',
                     password='qwer123$')

db_datamart = client['datamart']

# Engine(DataExtractor)

In [4]:
class DataExtractor():

    def __init__(self, host_ip, DBname, port, username, password, email_id, email_pw, client):
        self._phase = set()
        self._has_nosql = False
        self.client = client
        self.connect(host_ip, DBname, port, username, password)
        self.connect_email(email_id, email_pw)
        self.email_id = email_id

    def connect_email(self, email_id, email_pw):
        self.smtp = SMTP_SSL('smtp.gmail.com', 465)
        self.smtp.login(email_id, email_pw)

    def connect(self, host_ip, DBname, port, username, password):

        conn = psycopg2.connect(f"host={host_ip} " + (
                                f"port={port} ") + (
                                f"user={username} ") + (
                                f"password={password} ") + (
                                f"dbname={DBname}"))
        self.curs = conn.cursor()

        self.db_datamart = self.client['datamart']

    def __setattr__(self, name, value):
        if name == "NOSQL":
            self._has_nosql = True
        elif isinstance(value, dict):
            self._phase.add(name)
        super().__setattr__(name, value)

    def load_table(self):
        for name in self._phase:
            SQL = self.__dict__[name]['SQL']
            columns = self.__dict__[name]['columns']
            try:
                self.curs.execute(SQL)
            except Exception as e:
                print(e)
                print('connection retry...')
                self.connect(self.host_ip,
                             self.DBname,
                             self.port,
                             self.username,
                             self.password)
                self.curs.execute(SQL)
            temp = DataFrame(self.curs)
            if temp.shape[0] >= 1:
                temp.columns = columns
            else:
                temp = DataFrame(columns=columns)
            self.__dict__[f'dataframe_{name}'] = temp
            print(f'Table : {name} Load Complete')

        if self._has_nosql:
            for name, value in self.__dict__['NOSQL'].items():
                collection = self.db_datamart[name]
                if isinstance(value['SQL'], list):
                    temp = DataFrame(collection.find(*value['SQL']))
                if isinstance(value['SQL'], dict):
                    temp = DataFrame(collection.find(value['SQL']))
                if any([True if i == 'columns' else False for i in value]):
                    if temp.shape[0] >= 1:
                        temp.columns = columns
                    else:
                        temp = DataFrame(columns=columns)
                self.__dict__[f'dataframe_{name}'] = temp
                print(f'NoSQLTable : {name} Load Complete')
        self.curs.close()
    def run(self):
        raise NotImplementedError()

    def send_email(self, FROM, contents_file, patch=None, filename = None):
        subject = pd.read_csv(contents_file)
        for _, i in subject.iterrows():
            msg = MIMEMultipart()
            msg["FROM"] = FROM
            msg["to"] = i['address']
            msg['subject'] = i['subject']
            message = i['contents']
            self.msg = msg
            if patch is not None:
                if isinstance(patch, str):
                    with open(patch, 'rb') as f:
                        filename = patch
                        attachment = MIMEApplication(f.read())
                        attachment["Content-Disposition"] = 'attachment; filename=" {}"'.format(
                            f"{Path(filename).name}")
                if isinstance(patch, (DataFrame, Series)):
                    output = BytesIO()
                    writer = pd.ExcelWriter(output, engine='xlsxwriter')
                    patch.to_excel(writer, sheet_name='Sheet1')
                    writer.save()
                    output.seek(0)
                    attachment = MIMEApplication(output.read(), name = f'{filename}.xlsx', _subtype = 'xlsx')
                    attachment["Content-Disposition"] = 'attachment; filename=" {}"'.format(f"{filename}.xlsx")
                msg.attach(attachment)
                msg.attach(MIMEText(message))
                self.smtp.sendmail(msg["FROM"], msg["to"], msg.as_string())

        if patch is None:
            raise Exception('Please attach file on e-mail')

    def reset_smtp(self, email_id, email_pw):
        self.smtp.close()
        self.connect_email(email_id, email_pw)

    def businessday_counter(time_now, day_num, holidays):
        next_month = time_now + relativedelta(months=1)
        time_from = str(time_now.year) + '-' + str(time_now.month) + '-01'
        time_from_to = str((pd.Timestamp(str(next_month.year) + '-' + str(next_month.month) + '-01') - \
                            relativedelta(days=1)).date())
        holidays.columns = ['date', 'dotw', 'holiday']
        holidays['date'] = holidays['date'].astype('datetime64')
        date_range = pd.date_range(time_from,
                                   time_from_to,
                                   freq='D')
        date_table = DataFrame({'date': date_range,
                                'weekday': [i.weekday() for i in date_range]})
        date_table = pd.merge(date_table,
                              holidays[['date', 'holiday']],
                              on='date',
                              how='left')
        date_table.loc[date_table['weekday'] == 5, 'holiday'] = '토요일'
        date_table.loc[date_table['weekday'] == 6, 'holiday'] = '일요일'
        date_table.loc[date_table['holiday'].isna(), 'countable'] = True
        date_table.loc[~date_table['holiday'].isna(), 'countable'] = False
        date_table_workday = date_table[date_table['date'] >= time_from].reset_index(drop=True)
        date_table_workday['cumsum'] = date_table_workday['countable'].astype('int').cumsum()
        business_day = date_table_workday[date_table_workday['cumsum'] == day_num]['date'].iloc[0]
        TF = (business_day == pd.Timestamp(time_now))

        return TF, str(business_day.date())

# 마케팅

## 월간바이어리포트

In [4]:
class ETLBuyerDC(DataExtractor):

    def __init__(self, host_ip, DBname, port, username, password, email_id, email_pw, from_date, to_date, client):
        super().__init__(host_ip, DBname, port, username, password, email_id, email_pw, client)
        self.from_date = from_date
        self.to_date = to_date

    def load_table(self):
        self.NOSQL = {'dm_buyer' : {'SQL' : {}},
                      'dm_orderseries' : {'SQL' : {}}}

        super().load_table()

    def run(self):
        if not hasattr(self, 'dataframe_dm_orderseries'):
            self.load_table()

        marketing_dm = self.dataframe_dm_orderseries[['orderId',
                                                    'buyerId',
                                                    'createdAt',
                                                    'initTotalKRW',
                                                    'initHandlingFee',
                                                    'initQualityInspectionFee',
                                                    'initKRW',
                                                    'shippingCost',
                                                    'dcshippingCost',
                                                    'dcHandlingFee',
                                                    'dcQualityInspectionFee',
                                                    'productId',
                                                    'initSubtotal',
                                                    'initQuantity',
                                                    'brandId',
                                                    'building',
                                                    'countryCode',
                                                    'status_order']]

        buyers = self.dataframe_dm_buyer.copy()
        marketing_dm_kr = marketing_dm[(marketing_dm['countryCode'] == 'KR')]
        # 데이터 필터링
        marketing_dm_kr = pd.merge(marketing_dm_kr,
                                    Series([100, 200, 201, 202, 300, 400], name ='status_order'),
                                    on = 'status_order')
        marketing_dm_kr = marketing_dm_kr.drop(['status_order'], axis=1)
        # 대량 주문건 처리
        marketing_dm_kr = marketing_dm_kr[marketing_dm_kr['productId'].apply(lambda x: len(str(x).split('_')) <= 1)]
        # 열 삭제, 한글명 변경
        marketing_dm_kr.columns = ['orderId',
                                   '사업자ID',
                                   '구매날짜',
                                   '총 결제금액',
                                   '사입 수수료',
                                   '검수검품비',
                                   '상품 금액',
                                   '배송비',
                                   '배송비할인',
                                   '사입수수료할인',
                                   '검수검품비할인',
                                   '상품번호',
                                   '총금액',
                                   '수량',
                                   '브랜드ID',
                                   '상가',
                                   '국가코드']
        marketing_dm_kr['구매날짜'] = marketing_dm_kr['구매날짜'].dt.tz_localize('UTC').dt.tz_convert('Asia/Seoul')
        # 상품 주문번호를 주문번호 단위로 groupby
        temp = marketing_dm_kr.groupby('orderId')[['총 결제금액', '총금액', '사입 수수료', '사입수수료할인']].sum()
        # 상품 번호 단위를 상품 주문번호 단위로
        marketing_dm_buyer = pd.merge(marketing_dm_kr.drop(['총 결제금액', '사입 수수료', '사입수수료할인', '총금액'], axis=1),
                                      temp.reset_index(),
                                      on='orderId')
        #날짜 변경
        marketing_dm_buyer['구매날짜'] = marketing_dm_buyer['구매날짜'].apply(lambda x: pd.to_datetime(x).date())
        #dm_buyer와 결합
        marketing_dm_buyer = pd.merge(buyers[['buyerId', 'bizName']].rename({'buyerId': '사업자ID',
                                                                             'bizName': '사업자명'}, axis=1),
                                      marketing_dm_buyer,
                                      on='사업자ID',
                                      how='right')
        marketing_dm_buyer = marketing_dm_buyer[['orderId',
                                                '사업자ID',
                                                '사업자명',
                                                '구매날짜',
                                                '총 결제금액',
                                                '총금액',
                                                '사입 수수료',
                                                '배송비',
                                                '검수검품비',
                                                '사입수수료할인',
                                                '배송비할인',
                                                '검수검품비할인',
                                                '상품번호',
                                                '상품 금액',
                                                '수량',
                                                '브랜드ID',
                                                '상가']]
        # 총 결제금액에 배송비 합산
        marketing_dm_buyer['총 결제금액'] += marketing_dm_buyer['배송비']
        # 날짜 기준 절단
        marketing_dm_buyer = marketing_dm_buyer[(marketing_dm_buyer['구매날짜'].astype('str') >= self.from_date) & (
                marketing_dm_buyer['구매날짜'].astype('str') < self.to_date)]
        marketing_dm_buyer = marketing_dm_buyer.reset_index(drop=True)
        self.marketing_dm = marketing_dm_buyer.copy()

In [3]:
buyerdc = ETLBuyerDC('restore.cy1gcw3mvrbf.ap-northeast-2.rds.amazonaws.com',
                  'linkshops',
                  '5432',
                  'linkshops',
                  '$dpdlvmflf4',
                  'ji.kwon@linkshops.com','cjswp25*',
                   '2022-09-01', '2022-09-11',
                    client)

NameError: name 'client' is not defined

In [4]:
buyerdc.run()

NameError: name 'buyerdc' is not defined

In [1]:
buyerdc.marketing_dm_buyer

NameError: name 'buyerdc' is not defined

In [52]:
buyerdc.send_email('linkshops_bigdata', 
                   './contents.csv', 
                   buyerdc.marketing_dm_buyer, 
                   '마케팅_개인화구매리포트_20220822')

## 주간구매리포트

In [43]:
time_now = pd.Timestamp(datetime.datetime.now(), tz='UTC').tz_convert('Asia/Seoul')
time_from = \
'-'.join(str(datetime.datetime.strptime(str(time_now.date()), '%Y-%m-%d')).split(' ')[0].split('-')[0:2]) + \
'-01'
time_from_to = str(time_now.date())

In [44]:
class ETLBuyingReport(DataExtractor):

    def __init__(self, host_ip, DBname, port, username, password, email_id, email_pw, from_date, to_date, client):
        super().__init__(host_ip, DBname, port, username, password, email_id, email_pw, client)
        self.from_date = from_date
        self.to_date = to_date

    def load_table(self, demo_test=False):

        self.NOSQL = {'dm_orderseries' : {'SQL' : {}},
                      'dm_buyer' : {'SQL' : {}}}

        super().load_table()

    def run(self):
        
        if not hasattr(self, 'dm_orderseries'):
            self.load_table()
        dataframe_orderseries = self.dataframe_dm_orderseries.copy()
        dataframe_tel = self.dataframe_dm_buyer.copy()
        marketing_dm = dataframe_orderseries[['buyerId',
                                'orderId', 
                               'createdAt',
                               'productId',
                               'initTotalKRW',
                               'initSubtotal',
                               'initHandlingFee', 
                               'initQualityInspectionFee',
                               'shippingCost',
                               'dcHandlingFee',
                               'dcQualityInspectionFee',
                               'dcshippingCost',
                              'countryCode',
                              'status_order']]
        # 데이터 필터링
        marketing_dm_kr = marketing_dm[(marketing_dm['countryCode'] == 'KR')]
        marketing_dm_kr = pd.merge(marketing_dm_kr,
                             Series([100,200,201,202,300,400], name = 'status_order'),
                             on = 'status_order')
        marketing_dm_kr = marketing_dm_kr.drop(['status_order'], axis = 1)
        marketing_dm_kr = marketing_dm_kr[marketing_dm_kr['productId'].apply(lambda x : len(str(x).split('_')) <= 1)]
        marketing_dm_kr = marketing_dm_kr.drop(['productId'], axis = 1)
        # 컬럼명 한글 변환
        marketing_dm_kr.columns = ['바이어 아이디',
                                   '오더 아이디',
                                   '결제일',
                                   '총 결제금액',
                                   '상품금액',
                                   '사입비',
                                   '검수비',
                                   '배송비',
                                   '사입비할인',
                                   '검수비할인',
                                   '배송비할인',
                                   '국가코드']
        marketing_dm_kr['결제일'] = marketing_dm_kr['결제일'].dt.tz_localize('Asia/Seoul')
        marketing_dm_kr = marketing_dm_kr[(marketing_dm_kr['결제일'] >= self.from_date) & (
                                        marketing_dm_kr['결제일'] < self.to_date)]
        marketing_dm_kr['결제일'] = marketing_dm_kr['결제일'].apply(lambda x : pd.to_datetime(x).date())
        # 상품 주문번호를 주문번호 단위로 groupby
        marketing_dm_kr = pd.merge(marketing_dm_kr.groupby(['바이어 아이디',
                                                            '오더 아이디',
                                                            '결제일'])[['총 결제금액',
                                                                        '상품금액',
                                                                        '사입비',
                                                                        '검수비',
                                                                        '사입비할인',
                                                                        '검수비할인']].sum().reset_index(),
                                   # 상품 번호 단위를 상품 주문번호 단위로
                                     marketing_dm_kr[['바이어 아이디',
                                                      '오더 아이디',
                                                      '국가코드',
                                                      '배송비',
                                                      '배송비할인']].drop_duplicates().reset_index(drop = True),
                                    on = ['바이어 아이디', '오더 아이디'],
                                    how = 'right')
        # 배송비 합산
        marketing_dm_kr['총 결제금액'] += marketing_dm_kr['배송비'].fillna(0)
        # 바이어 정보 결합
        marketing_dm_kr = pd.merge(dataframe_tel[['buyerId','phoneNum','bizName']].rename({'buyerId' : '바이어 아이디',
                                                             'phoneNum' : '전화번호',
                                                             'bizName' : '주문자명'}, axis = 1),
                                     marketing_dm_kr,
                                     on = '바이어 아이디',
                                      how = 'right')
        self.marketing_dm = marketing_dm_kr.copy()

In [45]:
buyingreport = ETLBuyingReport('172.32.1.126',
                  'LINKSHOPS-20211121',
                  '5432',
                  'linkshops',
                   '$dpdlvmflf4',
                  'ji.kwon@linkshops.com','cjswp25*',
                   time_from, time_from_to,
                    client)

In [46]:
buyingreport.run()

NoSQLTable : dm_orderseries Load Complete
NoSQLTable : dm_buyer Load Complete


### 값검증

In [27]:
test = pd.read_csv('../데이터 추출/마케팅_테스트_20220901_20220911.csv')

In [28]:
pd.merge(test,
        Series(list(set(test['오더 아이디']).difference(buyingreport.marketing_dm_kr['오더 아이디'])), name = '오더 아이디'),
         on = '오더 아이디')['결제일'].unique()

array(['2022-09-01', '2022-09-11'], dtype=object)

In [31]:
buyingreport.marketing_dm_kr[buyingreport.marketing_dm_kr['주문자명'].isna()]

바이어 아이디         전화번호  주문자명   오더 아이디         결제일    총 결제금액      상품금액  \
256   159213   4048241115  None  1379496  2022-09-04  273540.0  232400.0   
440     1226  01073459866  None  1379108  2022-09-03   84310.0   66000.0   
508   147855          NaN   NaN  1380807  2022-09-06  662460.0  583000.0   
668   165335    411469919  None  1381134  2022-09-07  325100.0  273000.0   
682   147855          NaN   NaN  1381318  2022-09-08  118170.0  104000.0   
875   147855          NaN   NaN  1381432  2022-09-08   38630.0   34000.0   
879   148916    822459546  None  1381065  2022-09-07  129360.0  106000.0   

         사입비      검수비  사입비할인    검수비할인 국가코드     배송비   배송비할인  
256   8440.0   5500.0    0.0      0.0   KR  3960.0     0.0  
440   2390.0   1400.0    0.0      0.0   KR  7920.0     0.0  
508  21160.0      0.0    0.0  19100.0   KR     0.0  7920.0  
668   9900.0  14900.0    0.0      0.0   KR     0.0  3960.0  
682   3770.0      0.0    0.0   4800.0   KR     0.0  7920.0  
875   1230.0      0.0    0.0    500.0   KR     0.0  3960.0  
879   3840.0   1000.0    0.0      0.0   KR  7920.0     0.0

In [14]:
test2 = pd.merge(buyingreport.dataframe_dm_orderseries,
         Series(list(set(buyingreport.marketing_dm_kr['오더 아이디']).difference(test['오더 아이디'])), name = 'orderId').astype('int64'),
         on = 'orderId')['status_order']

In [15]:
test2.unique()

array([300], dtype=int64)

## 월간셀러리포트

In [ ]:
class ETLSeller(DataExtractor):

    def __init__(self, host_ip, DBname, port, username, password, email_id, email_pw, from_date, to_date, client):
        super().__init__(host_ip, DBname, port, username, password, email_id, email_pw, client)
        self.from_date = from_date
        self.to_date = to_date

    def load_table(self):
        self.NOSQL = {'dm_seller' : {'SQL' : {}},
                      'dm_buyer' : {'SQL' : {}},
                      'dm_orderseries' : {'SQL' : {}},
                      }
        super().load_table()

    def run(self):
        def type_extractor(x, types, recursive=None):
            try:
                if recursive is None:
                    return x[0][types]
                else:
                    return type_extractor([x[0][types]],
                                          types=recursive)
            except:
                return 'NaN'

        if not hasattr(self, 'dataframe_dm_orderseries'):
            self.load_table()

        marketing_dm_seller = self.dataframe_dm_orderseries.copy()
        dataframe_seller = self.dataframe_dm_seller.copy()
        dataframe_users = self.dataframe_dm_buyer.copy()
        # 데이터 필터링
        dataframe_users = dataframe_users[['buyerId','roles']]
        marketing_dm_seller = marketing_dm_seller[['brandId',
                                                   'initTotalKRW',
                                                   'initSubtotal',
                                                   'initHandlingFee',
                                                   'initTax',
                                                   'shippingCost',
                                                   'initQualityInspectionFee',
                                                   'productId',
                                                   'initKRW',
                                                   'initQuantity',
                                                   'buyerId',
                                                   'createdAt',
                                                   'countryCode',
                                                   'orderId',
                                                   'status_order']]
        marketing_dm_seller = pd.merge(marketing_dm_seller,
                                       Series([100, 200, 201, 202, 300, 400], name='status_order'),
                                       on='status_order')
        marketing_dm_seller = marketing_dm_seller.drop(['status_order'], axis=1)
        # 대량 주문건 처리
        ind = marketing_dm_seller['productId'].apply(lambda x : len(str(x).split('_')) > 1)
        # 대량 주문건 보정
        marketing_dm_seller.loc[ind,'initSubtotal'] = \
        marketing_dm_seller.loc[ind,'initTotalKRW']
        marketing_dm_seller.loc[ind, 'initTotalKRW'] = \
        marketing_dm_seller.loc[ind]['initSubtotal'].fillna(0) + \
        marketing_dm_seller.loc[ind]['initTax'].fillna(0) + \
        marketing_dm_seller.loc[ind]['initHandlingFee'].fillna(0)
        marketing_dm_seller = marketing_dm_seller.drop(['initTax'], axis = 1)
        # 셀러 정보 join
        marketing_dm_seller = pd.merge(marketing_dm_seller,
                                       dataframe_seller[['brandId',
                                                         'membership',
                                                         'membershipGrade']],
                                       on='brandId',
                                       how = 'left')
        marketing_dm_seller = pd.merge(marketing_dm_seller,
                                       dataframe_users,
                                       on='buyerId')
        marketing_dm_seller.loc[marketing_dm_seller['roles'] == 'NaN', 'roles'] = '탈퇴'
        marketing_dm_seller.columns = ['브랜드ID',
                                       '총 결제금액',
                                       '총금액',
                                       '사입수수료',
                                       '배송비',
                                       '검수검품비',
                                       '상품번호',
                                       '상품금액',
                                       '수량',
                                       '사업자ID',
                                       '구매일자',
                                       '국가코드',
                                       '주문번호',
                                       '글로벌셀러/베이직셀러',
                                       '멤버십등급',
                                       '빅바이어/일반바이어/탈퇴',
                                       ]

        marketing_dm_seller = pd.merge(marketing_dm_seller.drop(['총 결제금액', '총금액', '사입수수료'], axis=1),
                                       marketing_dm_seller.groupby('주문번호')[
                                           ['총 결제금액', '총금액', '사입수수료']].sum().reset_index(),
                                       on='주문번호')
        # 셀러 멤버십 처리
        marketing_dm_seller.loc[marketing_dm_seller['글로벌셀러/베이직셀러'] == 1, '글로벌셀러/베이직셀러'] = '글로벌셀러'
        marketing_dm_seller.loc[marketing_dm_seller['글로벌셀러/베이직셀러'] == 0, '글로벌셀러/베이직셀러'] = '베이직셀러'
        marketing_dm_seller.loc[marketing_dm_seller['멤버십등급'].apply(lambda x : len(str(x)) == 0),
                                '멤버십등급'] = '일반'
        # 날짜 기준 절단
        marketing_dm_seller['구매일자'] = marketing_dm_seller['구매일자'].dt.tz_localize('UTC').dt.tz_convert('Asia/Seoul')
        marketing_dm_seller = marketing_dm_seller[(marketing_dm_seller['구매일자'] >= self.from_date) & (
                marketing_dm_seller['구매일자'] < self.to_date)]
        marketing_dm_seller['구매일자'] = marketing_dm_seller['구매일자'].apply(lambda x: pd.to_datetime(x).date())
        # 총 결제금액에 배송비 합산
        marketing_dm_seller['총 결제금액'] += marketing_dm_seller['배송비']
        self.marketing_dm = marketing_dm_seller.copy()

In [14]:
time_now = pd.Timestamp(datetime.datetime.now(), tz='UTC')
time_from = str(datetime.datetime.strptime(str(time_now.date()), '%Y-%m-%d') - relativedelta(days = 1))
time_from_to = str(time_now.date())

In [15]:
time_from = '2022-11-01 00:00:00'
time_from_to = '2022-12-01 00:00:00'

In [16]:
dm_extractor = ETLSeller('restore.cy1gcw3mvrbf.ap-northeast-2.rds.amazonaws.com',
                              'linkshops',
                              '5432',
                              'linkshops',
                              '$dpdlvmflf4',
                              'ji.kwon@linkshops.com','cjswp25*',
                               time_from, time_from_to,
                                client)

In [17]:
dm_extractor.run()

NoSQLTable : dm_seller Load Complete
NoSQLTable : dm_buyer Load Complete
NoSQLTable : dm_orderseries Load Complete


In [20]:
dm_extractor.marketing_dm['멤버십등급'].value_counts()

일반       57263
black    10461
Name: 멤버십등급, dtype: int64

In [15]:
test = pd.read_excel('../데이터 추출/마케팅/마케팅_셀러_2022-10-12 00_00_00_2022-10-13.xlsx')

In [20]:
test_2 = pd.merge(dm_extractor.marketing_dm,
         test,
         on = '주문번호')

In [9]:
current_dir = 'C:/Users/Administrator/Documents/data_extraction_advanced'

In [250]:
dm_extractor.send_email('linkshops_bigdata',
                        f'{current_dir}/email_address/email_DailySeller.csv',
                        dm_extractor.marketing_dm,
                        f'마케팅_셀러_{time_from}_{time_from_to}')

In [10]:
dm_extractor.marketing_dm

브랜드ID     배송비  검수검품비     상품번호     상품금액   수량   사업자ID        구매일자 국가코드  \
228    16848  7920.0  700.0  7019969  26000.0  1.0  100864  2022-10-11   KR   
229    18340  7920.0  700.0  7008813  11000.0  1.0  100864  2022-10-11   KR   
230    18340  7920.0  700.0  7008813  11000.0  1.0  100864  2022-10-11   KR   
281     9202  3960.0  700.0  6641241  25000.0  1.0  134841  2022-10-11   KR   
606    10238  7920.0  700.0  6953717  20000.0  1.0  146865  2022-10-11   KR   
...      ...     ...    ...      ...      ...  ...     ...         ...  ...   
70970  17651     0.0    0.0  7030238  27000.0  1.0  131190  2022-10-11   KR   
70999  17862  3960.0  500.0  6999549  73000.0  1.0   20412  2022-10-11   KR   
71007  18653     0.0    0.0  6898709   6000.0  1.0   61683  2022-10-11   KR   
71008   6193  3960.0  500.0  6625974  84000.0  1.0  118390  2022-10-11   KR   
71030  17952  3960.0  700.0  7012774  39000.0  1.0  146251  2022-10-11   KR   

          주문번호 글로벌셀러/베이직셀러 빅바이어/일반바이어/탈퇴    총 결제금액       총금액    사입수수료  
228    1397944       글로벌셀러         buyer   64560.0   48000.0   1740.0  
229    1397944       글로벌셀러         buyer   64560.0   48000.0   1740.0  
230    1397944       글로벌셀러         buyer   64560.0   48000.0   1740.0  
281    1398557       베이직셀러         buyer   33070.0   25000.0    910.0  
606    1397974       베이직셀러         buyer  398380.0  325600.0  11800.0  
...        ...         ...           ...       ...       ...      ...  
70970  1398354       글로벌셀러         buyer   30680.0   27000.0    980.0  
70999  1398648       글로벌셀러         buyer   87410.0   73000.0   2650.0  
71007  1398730       베이직셀러         buyer    6820.0    6000.0    220.0  
71008  1398457       베이직셀러         buyer   99910.0   84000.0   3050.0  
71030  1397988       베이직셀러         buyer   48980.0   39000.0   1420.0  

[1543 rows x 15 columns]

In [228]:
dm_extractor.marketing_dm.to_excel('마케팅_셀러_20220901_20220923.xlsx', index = False)

## 린정 기획전

In [27]:
class ETLGlobalSKU(DataExtractor):
    def __init__(self, host_ip, DBname, port, username, password, email_id, email_pw, from_date, to_date, client):
        super().__init__(host_ip, DBname, port, username, password, email_id, email_pw, client)
        self.from_date = from_date
        self.to_date = to_date

    def load_table(self, demo_test=False):
        self.NOSQL = {'dm_merchandise': {'SQL': {
            'updatedAt_prod': {
                "$gte": dateutil.parser.parse(self.from_date)}}},
            'dm_seller': {'SQL': {}}}
        super().load_table()

    def run(self):
        def type_extractor(x, types, recursive=None):
            try:
                if recursive is None:
                    return x[0][types]
                else:
                    return type_extractor([x[0][types]],
                                          types=recursive)
            except:
                return 'NaN'

        def hasattribute(x, key, second_depth=None):
            if np.any(np.array(list(x.keys())) == key):
                if second_depth is not None:
                    if x[key] is not None:
                        return np.any(np.array(list(x[key].keys())) == second_depth)
                    else:
                        return False
                return True
            return False

        def making_material_info(x, prodid, varid):
            if x is not None:
                if len(x) > 0:
                    current = x['materials']
                    spec_ko = {i['name']['ko']: i['rate'] for i in current}

                    return DataFrame({'productId': prodid,
                                      'variantId': varid,
                                      'material_ko': [spec_ko]})

            return DataFrame({'productId': prodid,
                              'variantId': varid,
                              'material_ko': [None]})

        def making_specific_size(x, prodid, varid):
            if x is not None:
                if len(x) > 0:
                    base = x['rows']
                    spec_ko = [i['name']['ko'] for i in x['columns']]

                    size_ko = [{i['size']: [{k: j} if j is not None else None for j, k in zip(i['values'], spec_ko)]}
                               for i in base]

                    return DataFrame({'productId': prodid,
                                      'variantId': varid,
                                      'size_ko': [size_ko]})

            return DataFrame({'productId': prodid,
                              'variantId': varid,
                              'variantId': varid,
                              'size_ko': [None]})

        def making_washing_info(x, prodid, varid):
            if x is not None:
                if len(x) > 0:
                    current = x['washingInfos']
                    spec_ko = [i['name']['ko'] for i in current if i['checked']]

                    return DataFrame({'productId': prodid,
                                      'variantId': varid,
                                      'washinfo_ko': [spec_ko]})

            return DataFrame({'productId': prodid,
                              'variantId': varid,
                              'washinfo_ko': [None]})

        def making_fit_info(x, prodid, varid):
            if x is not None:
                if len(x) > 0:
                    current = x['fitInfos']
                    spec_ko = {i: j for i, j in zip([i['name']['ko'] for i in current],
                                                    [[j['name']['ko'] for j in i['items'] if j['checked']] for i in
                                                     current])}

                    return DataFrame({'productId': prodid,
                                      'variantId': varid,
                                      'fitinfo_ko': [spec_ko]})

            return DataFrame({'productId': prodid,
                              'variantId': varid,
                              'fitinfo_ko': [None]})

        if not hasattr(self, 'dataframe_dm_seller'):
            self.load_table()

        marketing_dm_merchandise = self.dataframe_dm_merchandise.copy()
        dataframe_seller = self.dataframe_dm_seller.copy()
        # 브랜드 정보 추출
        marketing_dm_merchandise['brandId'] = marketing_dm_merchandise['brand'].apply(
            lambda x: type_extractor([x], 'id'))
        marketing_dm_merchandise['brandName'] = marketing_dm_merchandise['brand'].apply(
            lambda x: type_extractor([x], 'name',
                                     recursive='ko'))

        marketing_dm_merchandise['building'] = marketing_dm_merchandise['brand'].apply(
            lambda x: type_extractor([type_extractor([type_extractor([x],
                                                                     'data',
                                                                     recursive='location')],
                                                     'building',
                                                     recursive='name')],
                                     'ko'))
        # paid Seller 추출
        marketing_dm_merchandise = pd.merge(dataframe_seller.loc[(dataframe_seller['membership'] == 1) & (
                dataframe_seller['membershipStatus'] == 200) & (dataframe_seller['membershipGrade'] == 'black'),
                                                                 'brandId'],
                                            marketing_dm_merchandise,
                                            on='brandId')
        # 소재 정보 추출(Json 까기)
        result_material_ko = marketing_dm_merchandise.apply(
            lambda x: making_material_info(x['SKUdata']['data']['specifications'],
                                           x['productId'],
                                           x['variantId'])
            if hasattribute(x['SKUdata']['data'],
                            'specifications',
                            'materials')
            else DataFrame({'productId': x['productId'],
                            'variantId': x['variantId'],
                            'material_ko': [None]}),
            axis=1)
        # 사이즈 정보 추출(Json 까기)
        result_size_ko = marketing_dm_merchandise.apply(
            lambda x: making_specific_size(x['SKUdata']['data']['measurement'],
                                           x['productId'],
                                           x['variantId'])
            if hasattribute(x['SKUdata']['data'],
                            'measurement')
            else DataFrame({'productId': x['productId'],
                            'variantId': x['variantId'],
                            'size_ko': [None]}),
            axis=1)
        # 세탁 정보 추출(JSon 까기)
        result_washing_ko = marketing_dm_merchandise.apply(
            lambda x: making_washing_info(x['SKUdata']['data']['specifications'],
                                          x['productId'],
                                          x['variantId'])
            if hasattribute(x['SKUdata']['data'],
                            'specifications',
                            'washingInfos')
            else DataFrame({'productId': x['productId'],
                            'variantId': x['variantId'],
                            'washinfo_ko': [None]}),
            axis=1)
        # 피팅정보 추출
        result_fit_ko = marketing_dm_merchandise.apply(
            lambda x: making_fit_info(x['SKUdata']['data']['specifications'],
                                      x['productId'],
                                      x['variantId'])
            if hasattribute(x['SKUdata']['data'],
                            'specifications',
                            'fitInfos')
            else DataFrame({'productId': x['productId'],
                            'variantId': x['variantId'],
                            'fitinfo_ko': [None]}),
            axis=1)
        # 인덱스 정보 통일 및 concat
        result_size_ko = pd.concat(result_size_ko.values).reset_index(drop=True)
        result_washing_ko = pd.concat(result_washing_ko.values).reset_index(drop=True)
        result_fit_ko = pd.concat(result_fit_ko.values).reset_index(drop=True)
        result_material_ko = pd.concat(result_material_ko.values).reset_index(drop=True)
        
        result_material_ko[['productId', 'variantId']] = result_material_ko[['productId', 'variantId']].astype('str')
        result_size_ko[['productId', 'variantId']] = result_size_ko[['productId', 'variantId']].astype('str')
        result_washing_ko[['productId', 'variantId']] = result_washing_ko[['productId', 'variantId']].astype('str')
        result_fit_ko[['productId', 'variantId']] = result_fit_ko[['productId', 'variantId']].astype('str')
        
        result_material_ko.index = result_material_ko['productId'] + ' - ' + result_material_ko['variantId']
        result_size_ko.index = result_size_ko['productId'] + ' - ' + result_size_ko['variantId']
        result_fit_ko.index = result_fit_ko['productId'] + ' - ' + result_fit_ko['variantId']
        result_washing_ko.index = result_washing_ko['productId'] + ' - ' + result_washing_ko['variantId']
    
        result_material_ko = result_material_ko.drop(['productId', 'variantId'], axis=1)
        result_size_ko = result_size_ko.drop(['productId', 'variantId'], axis=1)
        result_fit_ko = result_fit_ko.drop(['productId', 'variantId'], axis=1)
        result_washing_ko = result_washing_ko.drop(['productId', 'variantId'], axis=1)
        
        marketing_dm_merchandise.index = marketing_dm_merchandise['productId'].astype('str') + ' - ' + (
            marketing_dm_merchandise['variantId'].astype('str'))
        marketing_dm_merchandise = pd.concat([marketing_dm_merchandise,
                                              result_material_ko,
                                              result_size_ko,
                                              result_fit_ko,
                                              result_washing_ko],
                                             axis=1).reset_index(drop=True)
        # 1,3,4번 테이블 정오표(O / X) 변환
        marketing_dm_merchandise.loc[~marketing_dm_merchandise['material_ko'].isna(), 'material_xo'] = 'O'
        marketing_dm_merchandise.loc[marketing_dm_merchandise['material_ko'].isna(), 'material_xo'] = 'X'
        marketing_dm_merchandise.loc[~marketing_dm_merchandise['washinfo_ko'].isna(), 'washinfo_xo'] = 'O'
        marketing_dm_merchandise.loc[marketing_dm_merchandise['washinfo_ko'].isna(), 'washinfo_xo'] = 'X'
        marketing_dm_merchandise.loc[~marketing_dm_merchandise['fitinfo_ko'].isna(), 'fitinfo_xo'] = 'O'
        marketing_dm_merchandise.loc[marketing_dm_merchandise['fitinfo_ko'].isna(), 'fitinfo_xo'] = 'X'
        # 2번 테이블(사이즈 정보) 정오표(O / X) 변환
        marketing_dm_merchandise['size_xo_tf'] = marketing_dm_merchandise['size_ko'].apply(
            lambda x: [np.any(np.array([j for j in i.values()]) != None) for i in x][0]
            if x is not None
            else None)
        marketing_dm_merchandise.loc[marketing_dm_merchandise['size_xo_tf'].isna(), 'size_xo'] = 'X'
        marketing_dm_merchandise.loc[marketing_dm_merchandise['size_xo_tf'] == False, 'size_xo'] = 'X'
        marketing_dm_merchandise.loc[marketing_dm_merchandise['size_xo_tf'] == True, 'size_xo'] = 'O'
        # 각종 데이터 추출(Json 까기)
        marketing_dm_merchandise['updatedAt'] = marketing_dm_merchandise['SKUdata'].apply(lambda x: x['updatedAt'])
        marketing_dm_merchandise['createdAt'] = marketing_dm_merchandise['SKUdata'].apply(lambda x: x['createdAt'])
        marketing_dm_merchandise['names'] = marketing_dm_merchandise['SKUdata'].apply(lambda x: x['name']['ko'])
        marketing_dm_merchandise['categoryPath'] = marketing_dm_merchandise['SKUdata'].apply(
            lambda x: ' > '.join(type_extractor(type_extractor([x], 'data', recursive='categoryPath'), 'ko')))
        marketing_dm_merchandise = marketing_dm_merchandise[['productId',
                                                             'names',
                                                             'createdAt',
                                                             'updatedAt',
                                                             'brandId',
                                                             'brandName',
                                                             'building',
                                                             'categoryPath',
                                                             'size_xo',
                                                             'washinfo_xo',
                                                             'fitinfo_xo',
                                                             ]]
        # 한글명 변환
        marketing_dm_merchandise.columns = ['productId',
                                            '상품명',
                                            '등록날짜',
                                            '업데이트날짜',
                                            '브랜드 아이디',
                                            '브랜드명',
                                            '상가',
                                            '카테고리',
                                             '사이즈',
                                            '세탁정보',
                                            '피팅정보',
                                            ]
        marketing_dm_merchandise = marketing_dm_merchandise.drop_duplicates(
            subset=['productId']).reset_index(drop=True)
        # Time Zone 표준화
        marketing_dm_merchandise['등록날짜'] = marketing_dm_merchandise['등록날짜'].astype('datetime64').dt.tz_localize('UTC')
        marketing_dm_merchandise['등록날짜'] = marketing_dm_merchandise['등록날짜'].dt.tz_convert('Asia/Seoul')
        marketing_dm_merchandise['등록날짜'] = marketing_dm_merchandise['등록날짜'].apply(lambda x: pd.to_datetime(x).date())
        marketing_dm_merchandise['업데이트날짜'] = marketing_dm_merchandise['업데이트날짜'].astype('datetime64').dt.tz_localize(
            'UTC')
        marketing_dm_merchandise['업데이트날짜'] = marketing_dm_merchandise['업데이트날짜'].dt.tz_convert('Asia/Seoul')
        marketing_dm_merchandise['업데이트날짜'] = marketing_dm_merchandise['업데이트날짜'].apply(
            lambda x: pd.to_datetime(x).date())

        self.marketing_dm = marketing_dm_merchandise

In [28]:
time_now = pd.Timestamp(datetime.datetime.now(), tz='UTC')
time_from = str(datetime.datetime.strptime(str(time_now.date()), '%Y-%m-%d') - relativedelta(days=7)).split(' ')[0]
time_from_to = str(time_now.date())

In [29]:
globalsku = ETLGlobalSKU('restore.cy1gcw3mvrbf.ap-northeast-2.rds.amazonaws.com',
                  'linkshops',
                  '5432',
                  'linkshops',
                  '$dpdlvmflf4',
                  'ji.kwon@linkshops.com','cjswp25*',
                   '2022-09-19', '',
                    client)

In [31]:
globalsku.dataframe_dm_merchandise

Empty DataFrame
Columns: []
Index: []

In [30]:
globalsku.run()

NoSQLTable : dm_merchandise Load Complete
NoSQLTable : dm_seller Load Complete


KeyError: 'brand'

In [ ]:
test = pd.read_excel('../데이터 추출/마케팅/마케팅_글로벌SKU_2022-09-19_2022-09-26.xlsx')

In [15]:
globalsku.marketing_dm_merchandise.sort_values('업데이트날짜').to_excel('마케팅_글로벌셀러SKU_20220914_20220913.xlsx', index = False)

In [17]:
globalsku.send_email('linkshops_bigdata', 
                   '../data_extraction_advanced/email_address/email_GlobalSKU.csv', 
                   globalsku.marketing_dm_merchandise, 
                   f'마케팅_글로벌SKU_{"".join(time_from.split("-"))}')

### 값검증

In [41]:
test = pd.read_csv('test.csv')

In [42]:
test_2 = pd.merge(test,
        Series(list(set(test['브랜드 아이디']).difference(marketing_dm_merchandise['브랜드 아이디'])),
               name = '브랜드 아이디'),
                 on = '브랜드 아이디')

In [43]:
pd.merge(dataframe_seller,
Series(list(set(test['브랜드 아이디']).difference(marketing_dm_merchandise['브랜드 아이디'])),
       name = 'brandId'),
         on = 'brandId')

_id  sellerId               createdAt               updatedAt                       lastLoginAt  brandId                 name  status_brand  membership  membershipStatus  beforeMembership building floor flatLine flatNumber  sinceLastLogined  activeYn                                  membershipHistory            before_membership_date      before_membership_due  membership_term overshippedHistory  cumsumOvershipped  cusumOrderAmount  cusumPurchaseAmount  cusumOrderQuantity  cusumPurchaseQuantity  uploadedSku                                waitshipmentHistory  avgWaitShipment  avgWaitTerm
0  630da94d2621dbc92239c0e9     40252 2018-04-08 02:51:08.374 2022-08-29 22:08:19.419  2022-08-29 22:08:19.419959+00:00      617  POCLAIN jean(포크레인진)           100           1               100               0.0    Theot    4F        I         27                 0     False                                                 []                               NaN                        NaN              NaN                NaN                NaN         3125000.0             620000.0               138.0                   28.0         31.0  [{'17797060': {'wait_term': '3 days 22:45:00.1...         3.000000          3.0
1  630da94f2621dbc92239c951    136768 2021-05-03 22:25:47.475 2022-08-29 22:50:18.460  2022-08-29 22:50:18.460850+00:00    18092                  김패션           100           1               100               0.0    Theot    2F        D         20                 0     False                                                 []                               NaN                        NaN              NaN                NaN                NaN         4420000.0             568000.0               239.0                   34.0         28.0  [{'17760019': {'wait_term': '3 days 23:35:00.2...         4.428571          5.0
2  630da94f2621dbc92239ca0c    149922 2021-12-12 15:35:33.106 2022-08-29 23:59:27.694  2022-08-29 23:59:27.694588+00:00    18698                 크레파스           100           1               100               0.0      DWP    1F                  25                 0     False  [{'0 -> 1': {'historyUpdatedAt': '2022-05-26 0...  2022-05-26 01:54:03.428904+00:00  2022-06-12 14:00:00+00:00             17.0                NaN                NaN          513000.0             141000.0                49.0                   14.0         28.0                                                NaN              NaN          NaN
3  630da9542621dbc92239e1b0     53776 2018-08-21 12:12:27.033 2022-08-22 11:44:00.601  2022-08-22 11:44:00.601343+00:00    11841                어센틱서울           100           1               200               0.0      APM    4F                  04                 7     False                                                 []                               NaN                        NaN              NaN                NaN                NaN         1358000.0             277000.0                65.0                   13.0          NaN  [{'17733403': {'wait_term': '14 days 23:40:00....         8.000000          8.0

In [44]:
test_2

상품아이디  브랜드 아이디                           업뎃날짜                           생성날짜                                   상품명                              카테고리 상세 사이즈 핏정보 세탁정보
0   6953466    18698  2022-08-24 08:02:32.064 +0900  2022-08-21 02:40:55.339 +0900                 포니가디건-가을신상 기본 무지 캐시미어              ["여성", "아우터", "가디건"]      X   O    O
1   6934395    18698  2022-08-24 08:02:31.070 +0900  2022-08-09 20:10:32.107 +0900                무드원피스-가을신상 니트원피스 미니원피스           ["여성", "원피스/세트", "니트류"]      X   O    O
2   6916583    18698  2022-08-24 08:02:36.964 +0900  2022-07-22 09:46:50.591 +0900                         가을신상-스카시긴팔가디건              ["여성", "아우터", "가디건"]      O   O    O
3   6956812    18698  2022-08-26 04:26:36.695 +0900  2022-08-22 11:56:49.170 +0900                         블링알티-가을신상 꽈배기      ["여성", "상의", "니트", "라운드 니트"]      O   O    O
4   6916585    18698  2022-08-24 08:01:56.794 +0900  2022-07-22 09:49:13.464 +0900                       가을신상-이중랩가디건-투웨이              ["여성", "아우터", "가디건"]      O   O    O
5   6959811    18698  2022-08-26 04:26:38.107 +0900  2022-08-23 06:54:35.511 +0900                       앤드조끼-가을신상 베이직조끼           ["여성", "원피스/세트", "니트류"]      O   O    O
6   6963038    18698  2022-08-24 06:40:38.381 +0900  2022-08-24 06:40:38.381 +0900                             푸들알티-가을신상      ["여성", "상의", "니트", "라운드 니트"]      O   O    O
7   6658274    18698  2022-08-24 08:01:48.386 +0900  2022-03-08 07:13:12.354 +0900                           봄신상-면스판골지팬츠         ["바지", "여성", "하의", "니트류"]      X   X    X
8   6761053    18698  2022-08-26 04:26:19.012 +0900  2022-04-21 07:36:18.671 +0900                            301팬츠-밴딩팬츠         ["여성", "하의", "바지", "면바지"]      X   O    O
9   6703531    18698  2022-08-26 04:26:25.430 +0900  2022-03-28 07:13:43.588 +0900                              몽크조끼-봄신상      ["여성", "상의", "니트", "브이넥 니트"]      X   X    X
10  6761032    18698  2022-08-26 04:26:26.330 +0900  2022-04-21 07:23:30.854 +0900                            303팬츠-여름신상         ["여성", "하의", "바지", "면바지"]      X   O    O
11  6658250    18698  2022-08-26 04:26:29.812 +0900  2022-03-08 06:49:05.901 +0900                            봄신상-이중랩가디건              ["여성", "아우터", "가디건"]      X   X    X
12  6665770    18698  2022-08-26 04:26:33.346 +0900  2022-03-11 08:20:00.428 +0900                         마인크롭티-러블리,단가라         ["여성", "하의", "니트류", "바지"]      X   X    X
13  6959813    18698  2022-08-26 04:26:38.963 +0900  2022-08-23 06:57:01.907 +0900                            크레커알티-가을신상      ["여성", "상의", "니트", "라운드 니트"]      O   O    O
14  6729389    18092  2022-08-29 10:18:47.389 +0900  2022-04-07 07:25:19.567 +0900                          플리츠 롱 슬림 원피스  ["여성", "원피스/세트", "원피스", "롱 원피스"]      X   X    X
15  6954367    18092  2022-08-26 12:53:00.980 +0900  2022-08-21 19:57:12.697 +0900                  국내제작)다트 쮸르 하이웨스트 슬랙스         ["여성", "하의", "바지", "슬랙스"]      X   O    O
16  6339635    18092  2022-08-26 12:53:03.959 +0900  2021-10-04 01:22:54.589 +0900                국내제작) 쭈리 기모 숄더 포인트 원피스  ["여성", "원피스/세트", "원피스", "롱 원피스"]      X   O    O
17  6861278    18092  2022-08-26 12:53:11.907 +0900  2022-06-12 12:26:25.609 +0900                        신상)페이즐리 캉캉 스커트      ["여성", "하의", "스커트", "롱 스커트"]      X   O    O
18  6943144    18092  2022-08-26 12:53:08.778 +0900  2022-08-16 23:31:27.100 +0900                 fw신상)플리츠 머메이드 플레어 스커트      ["여성", "하의", "스커트", "롱 스커트"]      X   O    O
19  6939832    18092  2022-08-29 10:18:44.063 +0900  2022-08-15 23:48:13.431 +0900                    FW신상)세일러 카라배색 조거셋트            ["여성", "원피스/세트", "세트"]      X   O    O
20  6795823    18092  2022-08-29 10:18:49.071 +0900  2022-05-08 21:14:03.579 +0900                  (미송필수)언발 볼륨소매 반팔 맨투맨     ["여성", "상의", "티셔츠", "후드/맨투맨"]      X   O    O
21  6962374    18092  2022-08-26 12:52:49.520 +0900  2022-08-24 00:43:40.091 +0900                     국내제작)찰랑 언발 루즈핏 세트            ["여성", "원피스/세트", "세트"]      X   O    O
22  6962382    18092  2022-08-26 12:52:50.695 +0900  2022-08-24 00:45:57.

In [48]:
test[test['상품아이디'] == 6132574]

상품아이디  브랜드 아이디                           업뎃날짜                           생성날짜                                   상품명                       카테고리 상세 사이즈 핏정보 세탁정보
5756  6132574    11841  2022-08-24 23:13:32.507 +0900  2021-06-25 21:22:36.279 +0900  주문폭주) 아일렛 카고 스트링 팬츠 (가을신상 낱장가능 남여공용)  ["남성", "하의", "바지", "면바지"]      X   X    X

In [49]:
dataframe_merchandise[dataframe_merchandise['productId'] == 6132574]

Empty DataFrame
Columns: [_id, productId, variantId, date, totalKRW, finalTotalKRW, finalQuantity, quantity, SKUdata, variantData, brand, createdAt, isBulkOrder]
Index: []

In [47]:
dataframe_seller[dataframe_seller['brandId'] == 18698]

_id  sellerId               createdAt               updatedAt                       lastLoginAt  brandId  name  status_brand  membership  membershipStatus  beforeMembership building floor flatLine flatNumber  sinceLastLogined  activeYn                                  membershipHistory            before_membership_date      before_membership_due  membership_term overshippedHistory  cumsumOvershipped  cusumOrderAmount  cusumPurchaseAmount  cusumOrderQuantity  cusumPurchaseQuantity  uploadedSku waitshipmentHistory  avgWaitShipment  avgWaitTerm
5044  630da94f2621dbc92239ca0c    149922 2021-12-12 15:35:33.106 2022-08-29 23:59:27.694  2022-08-29 23:59:27.694588+00:00    18698  크레파스           100           1               100               0.0      DWP    1F                  25                 0     False  [{'0 -> 1': {'historyUpdatedAt': '2022-05-26 0...  2022-05-26 01:54:03.428904+00:00  2022-06-12 14:00:00+00:00             17.0                NaN                NaN          513000.0             141000.0                49.0                   14.0         28.0                 NaN              NaN          NaN

In [63]:
test_2 = pd.merge(test,
        Series(list(set(test['상품아이디']).difference(marketing_dm_merchandise['productId'])),
               name = '상품아이디'),
                 on = '상품아이디')

In [64]:
test_2['생성날짜'].astype('datetime64').dt.date.unique()

array([datetime.date(2022, 8, 23), datetime.date(2022, 8, 25),
       datetime.date(2022, 8, 29), datetime.date(2022, 8, 30)],
      dtype=object)

In [65]:
pd.Timestamp(test_2.iloc[0]['생성날짜'], tz = 'Asia/Seoul').tz_convert('UTC')

Timestamp('2022-08-23 15:54:06.235000+0000', tz='UTC')

In [66]:
pd.merge(test.rename({'상품아이디' : 'productId'}, axis = 1),
         marketing_dm_merchandise,
         on = 'productId')['등록날짜'].max()

'2022-08-28T15:15:41.509377+00:00'

In [52]:
dataframe_merchandise[dataframe_merchandise['productId'] == 6939885]

Empty DataFrame
Columns: [_id, productId, variantId, date, totalKRW, finalTotalKRW, finalQuantity, quantity, SKUdata, variantData, brand, createdAt, isBulkOrder]
Index: []

In [53]:
pd.Timestamp('2022-08-22 16:30:00', tz = 'UTC').tz_convert('Asia/Seoul')

Timestamp('2022-08-23 01:30:00+0900', tz='Asia/Seoul')

In [54]:
(test['생성날짜'].astype('datetime64') + pd.Timedelta(9, unit = 'h')).dt.date

0        2022-08-04
1        2022-08-04
2        2022-08-16
3        2022-08-21
4        2022-08-16
            ...    
23518    2022-08-15
23519    2022-08-14
23520    2022-08-15
23521    2022-08-15
23522    2022-08-07
Name: 생성날짜, Length: 23523, dtype: object

In [45]:
(test_2['생성날짜'].astype('datetime64') + pd.Timedelta(9, unit = 'h')).dt.date.unique()

array([], dtype=object)

In [71]:
pd.Timestamp('2022-08-22' + ' 23:59:59')

Timestamp('2022-08-22 23:59:59')

In [9]:
globalseller.marketing_dm_seller

브랜드ID      배송비   검수검품비     상품번호     상품금액   수량   사업자ID        구매일자  \
223      5590   3960.0   500.0  6757196  15000.0  1.0   99289  2022-08-17   
224     19395   3960.0   500.0  6559407  29000.0  1.0   99289  2022-08-17   
225     13126   3960.0   500.0  6749480  50000.0  1.0   99289  2022-08-17   
226     13126   3960.0   500.0  6816722  95000.0  1.0   99289  2022-08-17   
227     16233   3960.0  1500.0  6624556   5000.0  3.0   99289  2022-08-17   
...       ...      ...     ...      ...      ...  ...     ...         ...   
247975  18379  99000.0   500.0  6600218  24000.0  1.0  160429  2022-08-19   
247976  18379  99000.0   500.0  6002681  29000.0  1.0  160429  2022-08-19   
247977  18379  99000.0   500.0  6674462  29000.0  1.0  160429  2022-08-19   
247978  18379  99000.0   500.0  6440814  32000.0  1.0  160429  2022-08-19   
247979  18379  99000.0   500.0  6440814  32000.0  1.0  160429  2022-08-19   

       국가코드     주문번호 글로벌셀러/베이직셀러 빅바이어/일반바이어 국가코드_배송지       총금액    사입수수료  
223      KR  1370294       글로벌셀러      buyer       KR  204000.0   7400.0  
224      KR  1370294       베이직셀러      buyer       KR  204000.0   7400.0  
225      KR  1370294       베이직셀러      buyer       KR  204000.0   7400.0  
226      KR  1370294       베이직셀러      buyer       KR  204000.0   7400.0  
227      KR  1370294       베이직셀러      buyer       KR  204000.0   7400.0  
...     ...      ...         ...        ...      ...       ...      ...  
247975   SG  1371592       글로벌셀러      buyer       SG  146000.0  12850.0  
247976   SG  1371592       글로벌셀러      buyer       SG  146000.0  12850.0  
247977   SG  1371592       글로벌셀러      buyer       SG  146000.0  12850.0  
247978   SG  1371592       글로벌셀러      buyer       SG  146000.0  12850.0  
247979   SG  1371592       글로벌셀러      buyer       SG  146000.0  12850.0  

[28693 rows x 15 columns]

## 주간글로벌셀러리포트

In [41]:
time_now = pd.Timestamp(datetime.datetime.now(), tz='UTC')
time_from = str(datetime.datetime.strptime(str(time_now.date()), '%Y-%m-%d') -relativedelta(weekday = MO(-2))).split(' ')[0]
time_from_to = str((time_now - relativedelta(weekday = SU(-1)) + relativedelta(days = 1)).date())

In [42]:
class ETLWeeklyGlobalSeller(DataExtractor):
    def __init__(self, host_ip, DBname, port, username, password, email_id, email_pw, from_date, to_date, client):
        super().__init__(host_ip, DBname, port, username, password, email_id, email_pw, client)
        self.from_date = from_date
        self.to_date = to_date

    def load_table(self, demo_test=False):
        self.NOSQL = {'dm_orderseries': {'SQL': {}},
                      'dm_seller': {'SQL': {}},
                      'dm_buyer' : {'SQL' : {}}}
        super().load_table()

    def run(self):
        def type_extractor(x, types, recursive=None):
            try:
                if recursive is None:
                    return x[0][types]
                else:
                    return type_extractor([x[0][types]],
                                          types=recursive)
            except:
                return 'NaN'
        if not hasattr(self, 'dataframe_dm_buyer'):
            self.load_table()
            
        marketing_dm_seller = self.dataframe_dm_orderseries.copy()
        dataframe_seller = self.dataframe_dm_seller.copy()
        dataframe_buyer = self.dataframe_dm_buyer.copy()
        
        marketing_dm_seller = pd.merge(marketing_dm_seller,
                               dataframe_buyer[['buyerId','countryCodeShipment']],
                               on = 'buyerId')
        
        tz_tftable = marketing_dm_seller['createdAt'].apply(
                                                    lambda x : (x.tz_localize('UTC') < pd.Timestamp(
                                                        self.to_date, tz='Asia/Seoul').tz_convert('UTC')) & (
                                                                x.tz_localize('UTC') >= pd.Timestamp(
                                                                    self.from_date, tz='Asia/Seoul').tz_convert('UTC')))
        marketing_dm_seller = marketing_dm_seller[tz_tftable]
        marketing_dm_seller = pd.merge(dataframe_seller.loc[(dataframe_seller['membership'] == 1) & (
                                dataframe_seller['membershipStatus'] == 200), 'brandId'],
                                 marketing_dm_seller,
                                 on = 'brandId')
        marketing_dm_seller = pd.merge(marketing_dm_seller,
                             Series([100,101,102,200,201,202,203,300,400], name = 'status_order'),
                             on = 'status_order')
        marketing_dm_seller = marketing_dm_seller.drop(['status_order'], axis = 1)
        marketing_dm_seller = marketing_dm_seller[['orderId', 
                                           'buyerId',
                                           'buyerName',
                                           'createdAt',
                                           'initSubtotal',
                                           'initQuantity',
                                           'initTotalKRW',
                                           'finalSubtotal',
                                           'finalQuantity',
                                           'finalTotalKRW',
                                           'brandId',
                                           'building',
                                          'countryCode',
                                           'countryCodeShipment']]
        marketing_dm_seller.columns = ['orderId',
                                       '사업자ID',
                                       '사업자명',
                                       '구매날짜',
                                       '매출',
                                       '주문량',
                                       '최초',
                                       '최종매출',
                                       '거래량',
                                       '최종',
                                       '브랜드ID',
                                       '상가',
                                       '국가코드',
                                       '국가코드_배송지'] 
        marketing_dm_seller.loc[marketing_dm_seller['매출'].isna(), '매출'] = (
        marketing_dm_seller.loc[marketing_dm_seller['매출'].isna(), '최초']) 
        marketing_dm_seller.loc[marketing_dm_seller['최종매출'].isna(), '최종매출'] = (
        marketing_dm_seller.loc[marketing_dm_seller['최종매출'].isna(), '최종'])
        
        marketing_dm_seller = pd.merge(marketing_dm_seller.groupby(['orderId',
                                     '브랜드ID'])[['매출','최종매출']].sum().reset_index(),
                                 marketing_dm_seller[['orderId',
                                                      '브랜드ID',
                                                      '구매날짜',
                                                      '국가코드',
                                                      '국가코드_배송지']].drop_duplicates(),
                                 on = ['orderId','브랜드ID'])

        marketing_dm_seller = marketing_dm_seller[['브랜드ID',
                                                   'orderId',
                                                   '구매날짜',
                                                   '매출',
                                                   '최종매출',
                                                   '국가코드',
                                                   '국가코드_배송지']]
        marketing_dm_seller.columns = ['브랜드ID','orderId','구매일자','매출','총매출','국가코드','국가코드_배송지']
        marketing_dm_seller['구매일자'] = marketing_dm_seller['구매일자'].dt.tz_localize('UTC').dt.tz_convert('Asia/Seoul')
        marketing_dm_seller['구매일자'] = marketing_dm_seller['구매일자'].apply(lambda x : pd.to_datetime(x).date())
        
        self.marketing_dm_seller = marketing_dm_seller.copy()


In [43]:
weeklyglobalseller = ETLWeeklyGlobalSeller('restore.cy1gcw3mvrbf.ap-northeast-2.rds.amazonaws.com',
                              'linkshops',
                              '5432',
                              'linkshops',
                              '$dpdlvmflf4',
                              'ji.kwon@linkshops.com','cjswp25*',
                               time_from, time_from_to,
                                client)

In [44]:
weeklyglobalseller.run()

NoSQLTable : dm_orderseries Load Complete
NoSQLTable : dm_seller Load Complete
NoSQLTable : dm_buyer Load Complete


In [46]:
weeklyglobalseller.marketing_dm_seller

브랜드ID  orderId        구매일자        매출      총매출 국가코드 국가코드_배송지
0      2186  1382080  2022-09-12   10000.0      0.0   TW       TW
1      7595  1382080  2022-09-12    6000.0      0.0   TW       TW
2      8664  1382080  2022-09-12    5000.0      0.0   TW       TW
3     11554  1382080  2022-09-12   13000.0  13000.0   TW       TW
4     12432  1382080  2022-09-12   14000.0  14000.0   TW       TW
...     ...      ...         ...       ...      ...  ...      ...
5533   6658  1385723  2022-09-18   28000.0  28000.0   KR       KR
5534  12287  1385723  2022-09-18   30000.0      0.0   KR       KR
5535  14318  1385723  2022-09-18   14000.0  14000.0   KR       KR
5536  19296  1385723  2022-09-18   35000.0      0.0   KR       KR
5537  19137  1385724  2022-09-18  125000.0  45000.0   KR       KR

[5538 rows x 7 columns]

In [20]:
weeklyglobalseller.send_email('linkshops_bigdata', 
                   '../data_extraction_advanced/email_address/email_WeeklyGlobal.csv', 
                   weeklyglobalseller.marketing_dm_seller, 
                   '마케팅_주간글로벌매출_20220905_20220911')

In [16]:
weeklyglobalseller.marketing_dm_seller['매출'].sum()

252609778.0

In [22]:
weeklyglobalseller.marketing_dm_seller.sort_values('구매일자').to_excel('마케팅_주간글로벌매출_20220905_20220911.xlsx', index = False)

## 신상/낱장기획전

In [7]:
class ETLMinimumAndNewer(DataExtractor):
    def __init__(self, host_ip, DBname, port, username, password, email_id, email_pw, from_date, unit_date, client):
        super().__init__(host_ip, DBname, port, username, password, email_id, email_pw, client)
        self.from_date = from_date
        self.unit_date = unit_date

    def load_table(self, demo_test=False):
        time_now = self.from_date
        time_from = str(datetime.datetime.strptime(str(time_now.date()), '%Y-%m-%d') - (
                        relativedelta(days = self.unit_date[0] + 1)))
        self.NOSQL = {'dm_merchandise': {'SQL': {'updatedAt_prod':{
                            "$gt": dateutil.parser.parse(time_from)}}},
                      'dm_seller' : {'SQL' : {}}}
        super().load_table()

    def run(self):
        def type_extractor(x, types, recursive=None):
            try:
                if recursive is None:
                    return x[0][types]
                else:
                    return type_extractor([x[0][types]],
                                          types=recursive)
            except:
                return 'NaN'
        if not hasattr(self, 'dataframe_dm_seller'):
            self.load_table()   
        dataframe_merchandise = self.dataframe_dm_merchandise.copy()
        dataframe_seller = self.dataframe_dm_seller.copy()
        # 멤버십 등급 추출
        ind = dataframe_seller['membershipGrade'].apply(lambda x : (len(str(x)) == 0) | (
                                                        Series(x).isna() == True))
        dataframe_seller.loc[np.where(ind)[0], 'membershipGrade'] = '일반'
        dataframe_seller['isglobalmembership'] = (dataframe_seller['membership'] == 1) & (
                                                    dataframe_seller['membershipStatus'] == 200)
        dataframe_seller['isglobalmembership'] = (dataframe_seller['membership'] == 1) & (
                                                  dataframe_seller['membershipStatus'] == 200)
        dataframe_seller.loc[dataframe_seller['isglobalmembership'] == False, 'isglobalmembership'] = '베이직'
        dataframe_seller.loc[dataframe_seller['isglobalmembership'] != '베이직', 'isglobalmembership'] = '글로벌'
        dm_marketing = dataframe_merchandise[['productId',
                                              'SKUdata',
                                              'brand',
                                              'createdAt_prod',
                                              'updatedAt_prod']]
        # 각종 데이터 추출(Json 까기)
        dm_marketing['brandId'] = dm_marketing['brand'].apply(
            lambda x : type_extractor([x], 'id'))
        dm_marketing['Name'] = dm_marketing['SKUdata'].apply(
            lambda x : type_extractor([x], 'name', recursive = 'ko'))
        #dm_seller / dm_merchandise간 Join
        dm_marketing = pd.merge(dm_marketing,
                                dataframe_seller[['brandId',
                                                  'name',
                                                  'building',
                                                  'status_brand',
                                                  'isglobalmembership',
                                                  'membershipGrade']].rename({'status_brand': 'brandStatus',
                                                                                 'isglobalmembership': 'basic/global'},
                                                                                axis=1),
                        on = 'brandId')
        # 각종 데이터 추출(Json 까기)
        dm_marketing['isminimumquantity'] = dm_marketing['SKUdata'].apply(
                                            lambda x : type_extractor([x], 'data', recursive = 'isMinimumQuantity'))
        
        dm_marketing['productStatus'] = dm_marketing['SKUdata'].apply(
                                            lambda x : type_extractor([x], 'status'))
        # 활성 상태 제품 추출
        dm_marketing = dm_marketing[(dm_marketing['brandStatus'] != 0) & (dm_marketing['productStatus'] == 100)]
        # 낱장 가능 상품 추출
        dm_marketing_minimum = dm_marketing[dm_marketing['isminimumquantity'] == False]
        # 신상 제품 추출
        time_now = pd.Timestamp(self.from_date, tz='UTC')
        time_from = str(datetime.datetime.strptime(str(time_now.date()), '%Y-%m-%d') - (
                                        relativedelta(days = self.unit_date[1] + 1)))
        dm_marketing_newseven = dm_marketing[dm_marketing['updatedAt_prod'] >= time_from].reset_index(drop = True)
        # 낱장 가능 상품 / 신상 제품 concat
        dm_marketing_minimum_newseven  = pd.concat([dm_marketing_minimum,
                                                    dm_marketing_newseven],
                                                    axis = 0)
        dm_marketing_minimum_newseven = dm_marketing_minimum_newseven.drop_duplicates(
                                            'productId', keep = 'first').reset_index(drop = True)
        dm_marketing_minimum_newseven.loc[dm_marketing_minimum_newseven['isminimumquantity'] == False, 
                                          'isminimumquantity'] ='O'
        dm_marketing_minimum_newseven.loc[dm_marketing_minimum_newseven['isminimumquantity'] != 'O', 
                                          'isminimumquantity'] = 'X'
        dm_marketing_minimum_newseven = dm_marketing_minimum_newseven[['productId',
                                                                       'Name',
                                                                       'createdAt_prod',
                                                                       'updatedAt_prod',
                                                                       'brandId',
                                                                       'name',
                                                                       'basic/global',
                                                                       'membershipGrade',
                                                                       'building',
                                                                       'isminimumquantity']]
        dm_marketing_minimum_newseven.columns = ['productId','상품명','등록날짜','업데이트날짜','브랜드id','브랜드명',
                                                '글로벌셀러/베이직셀러','멤버십그레이드','상가','낱장 가능']
        self.dm_marketing = dm_marketing_minimum_newseven.copy()

In [8]:
minimumandnewer = ETLMinimumAndNewer('restore.cy1gcw3mvrbf.ap-northeast-2.rds.amazonaws.com',
                              'linkshops',
                              '5432',
                              'linkshops',
                              '$dpdlvmflf4',
                              'ji.kwon@linkshops.com','cjswp25*',
                              '2022-12-20', [14,7],
                                client)

In [9]:
minimumandnewer.run()

NoSQLTable : dm_merchandise Load Complete
NoSQLTable : dm_seller Load Complete


C:\Users\Administrator\AppData\Local\Temp\2\ipykernel_2176\484863467.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dm_marketing['brandId'] = dm_marketing['brand'].apply(
C:\Users\Administrator\AppData\Local\Temp\2\ipykernel_2176\484863467.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dm_marketing['Name'] = dm_marketing['SKUdata'].apply(


In [13]:
minimumandnewer.send_email('linkshops_bigdata', 
                   '../data_extraction_advanced/email_address/email_MinimumAndNewer.csv', 
                   minimumandnewer.dm_marketing, 
                   '마케팅_신상낱장_20220915')

In [10]:
minimumandnewer.dm_marketing.to_excel('마케팅_신상낱장_20220916.xlsx', index = False)

## 땡처리 기획전

In [12]:
class ETLFireSales(DataExtractor):
    def __init__(self, host_ip, DBname, port, username, password, email_id, email_pw, from_date, unit_date, client):
        super().__init__(host_ip, DBname, port, username, password, email_id, email_pw, client)
        #self.from_date = from_date
        #self.unit_date = unit_date

    def load_table(self, demo_test=False):
        # 땡처리 대상 상품 추출
        self.product_sale = {'SQL' : "SELECT \"id\"," + (
                                      "\"name\"->'ko',") + (
                                      "\"status\",") + (
                                      "\"data\",") + (
                                      "\"brand\",") + (
                                      "\"createdAt\",") + (
                                      "\"updatedAt\"") + (
                                      " FROM products WHERE ") + (
                                      "(name->>'ko') like '%sale2023%'"),
                            'columns' : ['productId',
                                         'Name',
                                         'productStatus',
                                         'SKUdata',
                                         'brand',
                                         'createdAt_prod',
                                         'updatedAt_prod']}
        self.NOSQL = {'dm_seller' : {'SQL' : {}}}
        super().load_table()

    def run(self):
        def type_extractor(x, types, recursive=None):
            try:
                if recursive is None:
                    return x[0][types]
                else:
                    return type_extractor([x[0][types]],
                                          types=recursive)
            except:
                return 'NaN'
        if not hasattr(self, 'dataframe_dm_seller'):
            self.load_table()
            
        dataframe_product_sale = self.dataframe_product_sale.copy()
        dataframe_seller = self.dataframe_dm_seller.copy()
        # 멤버십 등급 추출
        ind = dataframe_seller['membershipGrade'].apply(lambda x : (len(str(x)) == 0) | (
                                                        Series(x).isna() == True))
        dataframe_seller.loc[np.where(ind)[0], 'membershipGrade'] = '일반'
        dataframe_seller['isglobalmembership'] = (dataframe_seller['membership'] == 1) & (
        dataframe_seller['membershipStatus'] == 200)
        dataframe_seller.loc[dataframe_seller['isglobalmembership'] == False, 'isglobalmembership'] = '베이직'
        dataframe_seller.loc[dataframe_seller['isglobalmembership'] != '베이직', 'isglobalmembership'] = '글로벌'
        dm_marketing = dataframe_product_sale[['productId',
                                               'Name',
                                              'SKUdata',
                                              'brand',
                                              'productStatus',
                                              'createdAt_prod',
                                              'updatedAt_prod']]
        # 각종 데이터 추출(Json 까기)
        dm_marketing['createdAt_prod'] = dm_marketing['createdAt_prod'].dt.tz_convert('Asia/Seoul')
        dm_marketing['updatedAt_prod'] = dm_marketing['updatedAt_prod'].dt.tz_convert('Asia/Seoul')
        dm_marketing['brandId'] = dm_marketing['brand'].apply(
                                    lambda x : type_extractor([x], 'id'))
        dm_marketing['isminimumquantity'] = dm_marketing['SKUdata'].apply(
                lambda x : type_extractor([x], 'isMinimumQuantity'))
        # dm_seller / dm_merchandise간 Join
        dm_marketing = pd.merge(dm_marketing,
                                dataframe_seller[['brandId',
                                                  'name',
                                                  'building',
                                                  'status_brand',
                                                  'isglobalmembership',
                                                  'membershipGrade']].rename({'status_brand': 'brandStatus',
                                                                              'isglobalmembership': 'basic/global'},
                                                                        axis=1),
                                on = 'brandId')

        # 활성 상태 제품 추출
        dm_marketing = dm_marketing[(dm_marketing['brandStatus'] != 0) & (dm_marketing['productStatus'] == 100)]
        dm_marketing_minimum_newseven = dm_marketing.drop_duplicates(
                                    'productId', keep = 'first').reset_index(drop = True)
        # 낱장 가능 여부 정오표(O / X) 변환
        dm_marketing_minimum_newseven.loc[dm_marketing_minimum_newseven['isminimumquantity'] == False, 
                                          'isminimumquantity'] ='O'
        dm_marketing_minimum_newseven.loc[dm_marketing_minimum_newseven['isminimumquantity'] != 'O', 
                                          'isminimumquantity'] = 'X'
        dm_marketing_minimum_newseven = dm_marketing_minimum_newseven[['productId',
                                                               'Name',
                                                               'createdAt_prod',
                                                               'updatedAt_prod',
                                                               'brandId',
                                                               'name',
                                                               'membershipGrade',
                                                               'building',
                                                               'isminimumquantity']]
        dm_marketing_minimum_newseven.columns = ['productId','상품명','등록날짜','업데이트날짜','브랜드id','브랜드명',
                                                '멤버십그레이드','상가','낱장 가능']
        dm_marketing_minimum_newseven['등록날짜'] = dm_marketing_minimum_newseven['등록날짜'].dt.date
        dm_marketing_minimum_newseven['업데이트날짜'] = dm_marketing_minimum_newseven['업데이트날짜'].dt.date
        self.dm_marketing = dm_marketing_minimum_newseven.copy()

In [13]:
firesales = ETLFireSales('restore.cy1gcw3mvrbf.ap-northeast-2.rds.amazonaws.com',
                              'linkshops',
                              '5432',
                              'linkshops',
                              '$dpdlvmflf4',
                              'ji.kwon@linkshops.com','cjswp25*',
                              '','',
                                client)

In [18]:
firesales.send_email('linkshops_bigdata', 
                   '../data_extraction_advanced/email_address/email_FireSales.csv', 
                   firesales.dm_marketing, 
                   '마케팅_땡처리_20220915')

In [14]:
firesales.run()

Table : product_sale Load Complete
NoSQLTable : dm_seller Load Complete


In [15]:
firesales.dm_marketing

productId                              상품명        등록날짜      업데이트날짜  \
0      7090518           sale2023미엘 울 트위드팬츠sale  2022-11-09  2023-01-16   
1      7095525           sale2023양털 가디건 자켓 sale  2022-11-14  2023-01-16   
2      6463810              sale2023울니트조거팬츠sale  2021-11-27  2023-01-16   
3      7102395        sale2023보들보들부틀스트링조거팬츠sale  2022-11-17  2023-01-16   
4      7022734            sale2023마르코체크밴딩팬츠sale  2022-09-28  2023-01-16   
..         ...                              ...         ...         ...   
516    7151621       sale2023#)1025 그레이쉬 기모 롱일자  2023-01-10  2023-01-14   
517    7151614        sale2023#)1026 양포켓 기모 보이핏  2023-01-10  2023-01-14   
518    7152261       sale2023#)1016 더블포켓 기모 보이핏  2023-01-11  2023-01-14   
519    7153488     sale23 sale2023 ) 자가드 니트 집업   2023-01-11  2023-01-15   
520    7153475  sale23 sale2023 ) 자가드 3G 니트 집업   2023-01-11  2023-01-15   

     브랜드id 브랜드명 멤버십그레이드      상가 낱장 가능  
0    15171  베어리   black  Nuzzon     O  
1    15171  베어리   black  Nuzzon     O  
2    15171  베어리   black  Nuzzon     O  
3    15171  베어리   black  Nuzzon     O  
4    15171  베어리   black  Nuzzon     O  
..     ...  ...     ...     ...   ...  
516  10965   스텝      일반     청평화     O  
517  10965   스텝      일반     청평화     O  
518  10965   스텝      일반     청평화     O  
519   6896  벨로우      일반     APM     O  
520   6896  벨로우      일반     APM     O  

[521 rows x 9 columns]

# 회계

## 월말 글로벌멤버십 가입 현황

In [54]:
class ETLGlobalMembership(DataExtractor):
    def __init__(self, 
                 host_ip,
                 DBname,
                 port, 
                 username,
                 password,
                 email_id, 
                 email_pw,
                 from_date,
                 to_date,
                 client,
                 subscribe):
        super().__init__(host_ip, DBname, port, username, password, email_id, email_pw, client)
        self.time_now = from_date
        self.to_date = to_date
        self.subscribe = subscribe

    def load_table(self, demo_test=False):
        self.NOSQL = {'dm_seller': {'SQL': {}}}
        super().load_table()

    def run(self, threshold_membership = 1):
        def type_extractor(x, types, recursive=None):
            try:
                if recursive is None:
                    return x[0][types]
                else:
                    return type_extractor([x[0][types]],
                                          types=recursive)
            except:
                return 'NaN'
            
        def preprocessing_exception(avgsubperiod_row, date_id, seller_subscribe):
            membershipTerm = date_id.copy()
            membershipTerm['brandId'] = avgsubperiod_row['brandId']
            membershipTerm['group'] = 0
            membershipTerm['historyUpdatedAt'] = np.nan
            membershipTerm['historyUpdatedAtShift'] = np.nan
            membershipTerm['historyUpdatedAtDeriv'] = np.nan
            seller_subscribe = pd.concat([seller_subscribe, membershipTerm])
            return seller_subscribe

        def generating_group_num(init=None):
            if init is None:
                num = 0
            else:
                num = init
            while True:
                num += 1
                yield num
                
        def processing_for_subscribe_date(x, subscribe_tape, date_id):
            seller_subscribe = DataFrame()
            for num, avgsubperiod_row in x.iterrows():
                date_frame = DataFrame([list(i.values())[0] for i in avgsubperiod_row['membershipHistory']])
                if date_frame.shape[0] > 0:
                    date_frame = pd.concat([date_frame, Series(subscribe_tape[num], name='isBounce')], axis=1)
                    date_frame = pd.concat([date_frame, Series([list(i.keys())[0] for i in avgsubperiod_row['membershipHistory']],
                                                           name = 'subscribe_type')], axis = 1)
                    date_frame['currentMembership'] = float((avgsubperiod_row['membership'] == 1) & (
                                                avgsubperiod_row['membershipGrade'] == 'black'))
                    date_frame = date_frame.sort_values('historyUpdatedAtShift')
                    date_frame['historyUpdatedAt'] = date_frame['historyUpdatedAt'].astype('datetime64')
                    date_frame.loc[date_frame['historyUpdatedAtShift'] == 'current', 'historyUpdatedAtShift'] = np.nan
                    date_frame['historyUpdatedAtShift'] = date_frame['historyUpdatedAtShift'].astype('datetime64[ns]')
                    #date_frame = date_frame[date_frame['isBounce'] == False].reset_index(drop=True)
                    date_frame['historyUpdatedAtShift'] = date_frame['historyUpdatedAtShift'].fillna(datetime.datetime.now())
                    date_frame_for_deriv = date_frame[date_frame['isBounce'] == False]
                    date_frame_for_deriv = date_frame_for_deriv.reset_index()
                    if date_frame_for_deriv.shape[0] > 1:
                        startDate_shift = Series(
                            date_frame_for_deriv.loc[1: date_frame_for_deriv.shape[0] - 1, 'historyUpdatedAt'].reset_index(drop=True),
                            name='startDate_shift')
                    elif date_frame_for_deriv.shape[0] == 1:
                        startDate_shift = Series(
                            date_frame_for_deriv.reset_index().loc[0, 'historyUpdatedAtShift'],
                            name='startDate_shift')
                    else:
                        seller_subscribe = preprocessing_exception(avgsubperiod_row, date_id, seller_subscribe)
                        continue
                    date_frame_for_deriv = pd.concat([date_frame_for_deriv, startDate_shift], axis=1)
                    if date_frame_for_deriv.loc[date_frame_for_deriv.shape[0] - 1, 'isBounce'] == False:
                        date_frame_for_deriv.loc[date_frame_for_deriv.shape[0] - 1, 'startDate_shift'] = (
                            date_frame_for_deriv.loc[date_frame_for_deriv.shape[0] - 1, 'historyUpdatedAtShift'])
                    if date_frame_for_deriv.shape[0] == 0:
                        seller_subscribe = preprocessing_exception(avgsubperiod_row, date_id, seller_subscribe)

                    TF_frame = DataFrame({'isSuccessive': (date_frame_for_deriv['startDate_shift'] - date_frame_for_deriv[
                                                              'historyUpdatedAtShift']) < (
                                                              pd.Timedelta(threshold_membership, unit='D')),
                                          'isNegative': (date_frame_for_deriv['startDate_shift'] - date_frame_for_deriv[
                                                         'historyUpdatedAtShift']) < (
                                                         pd.Timedelta(0, unit='D'))})
                    date_frame_for_deriv.index = date_frame_for_deriv['index']
                    TF_frame.index = date_frame_for_deriv['index']
                    date_frame_for_deriv = pd.concat([date_frame_for_deriv, TF_frame], axis=1)    
                    date_frame_for_deriv['isCurrentAndBounce'] = date_frame_for_deriv['isBounce'].isna()
                    date_frame_isConfirmed = date_frame[(date_frame['historyUpdatedAtDeriv'].isna()) #& (date_frame['isBounce'])
                                                       ]
                    date_frame_isConfirmed = date_frame_isConfirmed.reset_index()
                    TF_frame = DataFrame({'isConfirmed' : date_frame_isConfirmed['subscribe_type'].apply(
                                    lambda x : str(float(x.split(' -> ')[-1])) == \
                                               str(float(date_frame_isConfirmed['currentMembership'].values[0])))})
                    if date_frame_isConfirmed.shape[0] >= 1:
                        date_frame_isConfirmed.index = date_frame_isConfirmed['index']
                        TF_frame.index = date_frame_isConfirmed['index']
                    date_frame_isConfirmed = pd.concat([date_frame_isConfirmed, TF_frame], axis=1).reset_index(drop = True)
                    date_frame_temp = pd.concat([date_frame_for_deriv,
                                                       date_frame_isConfirmed], axis = 0)
                    date_frame_temp = date_frame_temp.drop_duplicates(subset = 'index', keep = 'last')
                    date_frame_temp['isCurrentAndBounce'] = date_frame_temp['historyUpdatedAtDeriv'].isna()
                    date_frame_temp[['isSuccessive','isNegative']] = \
                    date_frame_temp[['isSuccessive','isNegative']].fillna(method = 'ffill')
                    date_frame_temp['isConfirmed'] = date_frame_temp['isConfirmed'].fillna(False)
                    gen_group_num = generating_group_num()
                    tape = list()
                    group_num = 0
                    for i in date_frame_temp.iterrows():
                        if (i[1]['isSuccessive'] == False) | (
                            i[1]['isNegative'] == True) | (
                            i[1]['isCurrentAndBounce']):
                            group_num = next(gen_group_num)
                        tape.append(group_num)
                    date_frame_temp['group'] = tape
                    date_frame_temp.index = date_frame_temp['index']
                    date_frame_temp = date_frame_temp.drop(['index'], axis = 1)
                    date_frame = pd.concat([date_frame,
                                            date_frame_temp[['startDate_shift',
                                                             'isSuccessive',
                                                             'isNegative',
                                                             'isConfirmed',
                                                             'isCurrentAndBounce',
                                                             'group']]
                                           ],
                                           axis = 1)
                    date_frame['group'] = date_frame['group'].fillna(method = 'ffill')
                    date_frame = date_frame.groupby('group').apply(
                                lambda x : pd.concat([x.reset_index().loc[0, ['historyUpdatedAt']],
                                                      x.reset_index().loc[len(x) -1, ['historyUpdatedAtShift',
                                                                                    'isBounce',
                                                                                    'isConfirmed']]], axis = 0))
                    temp = date_frame[['historyUpdatedAt',
                                       'historyUpdatedAtShift']].astype('str').apply(
                                                                        lambda x: (x[1] >= date_id['date']) & (
                                                                                  x[0] <= date_id['date']),
                                                                                               axis = 1)
                    temp.columns = date_id['date']
                    temp.index = date_frame['historyUpdatedAtShift']
                    date_frame = pd.merge(date_frame,
                                          temp.stack().reset_index(name='isoverdate'),
                                          on=['historyUpdatedAtShift'])
                    date_frame = date_frame.sort_values(['date'])
                    date_frame['sellerId'] = avgsubperiod_row['sellerId']
                    date_frame['brandId'] = avgsubperiod_row['brandId']
                    date_frame.sort_values(['historyUpdatedAt',
                                            'historyUpdatedAtShift',
                                            'date'])
                    # historyUpdatedAtShift 값으로  sorting을 했기 때문에, -1은 historyUpdatedAtShift가 가장 마지막인 값을 의미한다
                    seller_subscribe = pd.concat([seller_subscribe, date_frame])
                else:
                    seller_subscribe = preprocessing_exception(avgsubperiod_row, date_id, seller_subscribe)
                    continue
            
            return seller_subscribe
                
        if not hasattr(self, 'dataframe_dm_seller'):
            self.load_table()
        
        dataframe_seller = self.dataframe_dm_seller.copy()
        dataframe_seller = dataframe_seller.dropna(subset = ['membershipHistory'])
        dataframe_seller = dataframe_seller[dataframe_seller['membershipHistory'].apply(lambda x : len(x) != 0)]
        
        date_id = DataFrame({'date' : pd.date_range(
                                                    pd.Timestamp('2022-05-01 00:00:00', tz = 'Asia/Seoul'), 
                                                    time_now
                                                    ).values
                            })
        date_id['date'] = date_id['date'].dt.tz_localize('UTC').dt.tz_convert('Asia/Seoul')
        
        # ----- normal ------ #
        isNormal = dataframe_seller['membershipHistory'].apply(lambda x : [list(i.keys()) for i in x][0][0] in self.subscribe)
        dataframe_seller_normal = dataframe_seller[isNormal].reset_index(drop = True)
        subscribe_tape = dataframe_seller_normal['membershipHistory'].apply(
                                    lambda x : [list(i.keys())[0] not in self.subscribe for i in x] 
                                                  if not isinstance(x, float) else None)
        ind = pd.concat([dataframe_seller_normal['membershipHistory'].apply(lambda x : [
                                                        list(i.values())[0]['historyUpdatedAtShift'] != 'current' 
                                                        for i in x][-1]
                                                        if len(x) > 0 else [False]),
                         subscribe_tape.apply(lambda x : x[-1] == False if len(x) > 0 else False)],
                         axis = 1).apply(lambda x : np.all(x), axis = 1)
        dataframe_seller_normal.loc[ind, 'membershipHistory'].apply(
                    lambda x : x.append({'1 -> 0' : {'historyUpdatedAt' : list(x[-1].values())[0]['historyUpdatedAtShift'],
                                                     'historyUpdatedAtShift' : 'current',
                                                     'historyUpdatedAtDeriv' : np.nan}}))
        subscribe_tape = dataframe_seller_normal['membershipHistory'].apply(
                                    lambda x : [list(i.keys())[0] not in self.subscribe for i in x] 
                                                  if not isinstance(x, float) else None)
        seller_subscribe_normal = processing_for_subscribe_date(dataframe_seller_normal,
                                                         subscribe_tape,
                                                         date_id)
        seller_subscribe_normal = pd.concat([# 과거 가입자
                            seller_subscribe_normal[(seller_subscribe_normal['isBounce'] == False) & (
                                              seller_subscribe_normal['isoverdate'] == True)],
                            # 이탈 신청 했으니 아직 미승인
                            seller_subscribe_normal[(seller_subscribe_normal['isBounce'] == True) & (
                                                    seller_subscribe_normal['isoverdate'] == True) & (
                                                         seller_subscribe_normal['isConfirmed'] == False)]],
                            axis = 0).reset_index(drop = True)
        # ------ type_2 ------- #
        isType2 = dataframe_seller['membershipHistory'].apply(lambda x : [list(i.keys()) for i in x][0][0] not in self.subscribe)
        dataframe_seller_type2 = dataframe_seller[isType2].reset_index(drop = True)
        subscribe_tape = dataframe_seller['membershipHistory'].apply(
                                    lambda x : [list(i.keys())[0] not in self.subscribe for i in x] 
                                                  if not isinstance(x, float) else None)
        ind = pd.concat([dataframe_seller_type2['membershipHistory'].apply(lambda x : [
                                                        list(i.values())[0]['historyUpdatedAtShift'] != 'current' 
                                                        for i in x][-1]
                                                        if len(x) > 0 else [False]),
                         subscribe_tape.apply(lambda x : x[-1] == False if len(x) > 0 else False)],
                         axis = 1).apply(lambda x : np.all(x), axis = 1)
        dataframe_seller_type2.loc[ind, 'membershipHistory'].apply(
                    lambda x : x.append({'1 -> 0' : {'historyUpdatedAt' : list(x[-1].values())[0]['historyUpdatedAtShift'],
                                                     'historyUpdatedAtShift' : 'current',
                                                     'historyUpdatedAtDeriv' : np.nan}}))
        subscribe_tape = dataframe_seller_type2['membershipHistory'].apply(
                            lambda x : [list(i.keys())[0] not in self.subscribe for i in x] 
                                          if not isinstance(x, float) else None)
        seller_subscribe_type2 = processing_for_subscribe_date(dataframe_seller_type2,
                                                                 subscribe_tape,
                                                                 date_id)
        seller_subscribe_type2 = pd.concat([# 과거 가입자
                            seller_subscribe_type2[(seller_subscribe_type2['isBounce'] == False) & (
                                                    seller_subscribe_type2['isoverdate'] == True)],
                            # 이탈 신청 했으니 아직 미승인
                            seller_subscribe_type2[(seller_subscribe_type2['isBounce'] == True) & (
                                                    seller_subscribe_type2['isoverdate'] == True) & (
                                                    seller_subscribe_type2['isConfirmed'] == False)]],
                            axis = 0).reset_index(drop = True)
        dataframe_seller_type4 = self.dataframe_dm_seller[(self.dataframe_dm_seller['membershipHistory'].isna()) & (
                                                self.dataframe_dm_seller['membership'] == 1) & (
                                                self.dataframe_dm_seller['membershipStatus'] == 200) & (
                                                self.dataframe_dm_seller['status_brand'] != 0)]
        dataframe_seller_type4 = dataframe_seller_type4[['sellerId','brandId']]
        dataframe_seller_type4['historyUpdatedAt'] = np.nan
        dataframe_seller_type4['date'] = pd.Timestamp(time_now.date(), tz = 'Asia/Seoul')
        
        # -----preprocessing-------#
        
        dm_account_global = pd.concat([seller_subscribe_normal,
                                       seller_subscribe_type2,
                                       dataframe_seller_type4])
        dm_account_global = pd.merge(dm_account_global,
                                     self.dataframe_dm_seller[['brandId',
                                                               'name',
                                                               'building',
                                                               'floor',
                                                               'flatLine',
                                                               'flatNumber',
                                                               'status_brand']],
                                    on = 'brandId')
        dm_account_global = dm_account_global[['historyUpdatedAt',
                                               'date',
                                               'sellerId',
                                               'brandId',
                                               'name',
                                               'building',
                                               'floor',
                                               'flatLine',
                                               'flatNumber',
                                               'status_brand']]
        dm_account_global.columns = ['가입일자',
                                     '날짜',
                                     '셀러ID',
                                     '브랜드ID',
                                     '브랜드명',                                   
                                     '상가명',
                                     '층',
                                     '열',
                                     '호',
                                     '브랜드상태']
        
        dm_account_global.loc[dm_account_global['브랜드상태'] == 100, '브랜드상태'] = '운영중'
        dm_account_global.loc[dm_account_global['브랜드상태'] == 0, '브랜드상태'] = '폐업'
        dm_account_global['날짜'] = dm_account_global['날짜'].dt.date
        self.dm_account = dm_account_global.copy()

In [55]:
time_now = pd.Timestamp(datetime.datetime.now(), tz='UTC').tz_convert('Asia/Seoul')

In [56]:
globalmembership = ETLGlobalMembership('restore.cy1gcw3mvrbf.ap-northeast-2.rds.amazonaws.com',
                              'linkshops',
                              '5432',
                              'linkshops',
                              '$dpdlvmflf4',
                              'ji.kwon@linkshops.com','cjswp25*',
                              str(time_now.date()), '',
                                client,
                                ['0 -> 1', '1 -> 2', '0 -> 2', '2 -> 1'])

In [57]:
globalmembership.run()

NoSQLTable : dm_seller Load Complete


C:\Users\Administrator\AppData\Local\Temp\2\ipykernel_8984\1140315070.py:160: FutureWarning: Behavior when concatenating bool-dtype and numeric-dtype arrays is deprecated; in a future version these will cast to object dtype (instead of coercing bools to numeric values). To retain the old behavior, explicitly cast bool-dtype arrays to numeric dtype.
  seller_subscribe = pd.concat([seller_subscribe, date_frame])


In [62]:
timetable = DataFrame({'startDate' : 
                       [str(i.date()).split('-')[0] + '-' + str(i.date()).split('-')[1] + '-01  00:00:00.000000 +0900' for i in pd.date_range(
                          start = '2022-05-01',
                          end = '2023-01-01', 
                          freq = 'M',
                         inclusive = 'both')],
                       'endDate' : 
                       [str(i.date()).split('-')[0] + '-' + str(i.date()).split('-')[1] + '-01  00:00:00.000000 +0900' for i in pd.date_range(
                          start = '2022-06-01',
                          end = '2023-02-01', 
                          freq = 'M',
                         inclusive = 'both')]})

In [110]:
globalmembership.dm_account[globalmembership.dm_account['날짜'] == pd.Timestamp('2022-12-30')]

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\ops\array_ops.py:73: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  result = libops.scalar_compare(x.ravel(), y, op)


가입일자          날짜      셀러ID    브랜드ID  \
65      2022-10-25 11:02:44.384727  2022-12-30   92416.0  16464.0   
236     2022-07-25 08:53:58.398072  2022-12-30   44576.0   5355.0   
422     2022-08-15 13:49:26.082101  2022-12-30  164463.0  19414.0   
619     2022-06-29 21:34:12.960741  2022-12-30   46903.0   8311.0   
761     2022-08-23 03:18:25.718811  2022-12-30   84950.0  14678.0   
...                            ...         ...       ...      ...   
102820         2022-05-08 00:00:00  2022-12-30       NaN  18907.0   
103070         2022-05-08 00:00:00  2022-12-30       NaN   5125.0   
103229  2022-10-06 13:39:54.795458  2022-12-30  106270.0  16441.0   
103588  2022-06-21 13:57:07.054923  2022-12-30   39840.0   8956.0   
104133  2022-07-07 02:09:19.664951  2022-12-30     633.0    633.0   

               브랜드명          상가명   층     열        호 브랜드상태  
65              리마쥬         제일평화  2F  None       55   운영중  
236             모티브          남평화  B1    신관       07   운영중  
422            베베스푼       원(남대문)  1F        172.173   운영중  
619            야무지게       Nuzzon  1F            701   운영중  
761              한나  ddp패션몰(유어스)  3F  None       23   운영중  
...             ...          ...  ..   ...      ...   ...  
102820         롤리폴리           기타      None     None   운영중  
103070         버블언더        Theot  4F                   폐업  
103229          아스타          남평화  3F            138   운영중  
103588     게임(GAME)          APM  4F             27   운영중  
104133  트리안(treean)        Theot  3F     J       04   운영중  

[447 rows x 10 columns]

In [112]:
nonhistory = pd.read_csv('회계_이력미존재멤버십셀러_202210.csv')

In [118]:
for _, i in timetable.iterrows():
    from_date = i['startDate']
    to_date = i['endDate']
    print(from_date, ' ', to_date, ' has started')
    temp = globalmembership.dm_account[(globalmembership.dm_account['날짜'] >= pd.Timestamp(from_date)) & (
                                        globalmembership.dm_account['날짜'] < pd.Timestamp(to_date))]
    temp = temp[['가입일자',
                 '셀러ID',
                 '브랜드ID',
                 '브랜드명',
                 '상가명',
                 '층',
                 '열',
                 '호',
                 '브랜드상태']].drop_duplicates(keep = 'last')
    month = i['startDate'].split('-')[0] + i['startDate'].split('-')[1]
    print(set(temp['브랜드ID']).intersection(nonhistory['브랜드ID']))
    temp.to_excel(f'회계_월별셀러멤버십가입현황_{month}.xlsx')

2022-05-01  00:00:00.000000 +0900   2022-06-01  00:00:00.000000 +0900  has started
set()
2022-06-01  00:00:00.000000 +0900   2022-07-01  00:00:00.000000 +0900  has started


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\ops\array_ops.py:73: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  result = libops.scalar_compare(x.ravel(), y, op)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\ops\array_ops.py:73: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  result = libops.scalar_compare(x.ravel(), y, op)


set()
2022-07-01  00:00:00.000000 +0900   2022-08-01  00:00:00.000000 +0900  has started


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\ops\array_ops.py:73: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  result = libops.scalar_compare(x.ravel(), y, op)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\ops\array_ops.py:73: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  result = libops.scalar_compare(x.ravel(), y, op)


set()
2022-08-01  00:00:00.000000 +0900   2022-09-01  00:00:00.000000 +0900  has started
set()
2022-09-01  00:00:00.000000 +0900   2022-10-01  00:00:00.000000 +0900  has started


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\ops\array_ops.py:73: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  result = libops.scalar_compare(x.ravel(), y, op)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\ops\array_ops.py:73: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  result = libops.scalar_compare(x.ravel(), y, op)


set()
2022-10-01  00:00:00.000000 +0900   2022-11-01  00:00:00.000000 +0900  has started
set()
2022-11-01  00:00:00.000000 +0900   2022-12-01  00:00:00.000000 +0900  has started


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\ops\array_ops.py:73: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  result = libops.scalar_compare(x.ravel(), y, op)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\ops\array_ops.py:73: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  result = libops.scalar_compare(x.ravel(), y, op)


set()
2022-12-01  00:00:00.000000 +0900   2023-01-01  00:00:00.000000 +0900  has started
set()


### 값검증

In [19]:
test = pd.read_excel('20221117_membership_.xlsx')

test = test.rename({'id' : 'brandId'}, axis = 1)

In [20]:
test_2 = globalmembership.dm_account[globalmembership.dm_account['날짜'] == str(time_now.date())]

test_2 = test_2.rename({'브랜드ID' : 'brandId'}, axis = 1)

In [21]:
pd.merge(test, 
         test_2,
         on = 'brandId')

brandId        브랜드명_x              상가  층_x  열_x    호수  \
0      18822        러브유어셀프     ddp패션몰(유어스)   1F  NaN    20   
1       6044  더블엠(DOUBLEM)           Theot   1F    E    13   
2      14132     블랙핑크_유아동복          혜양엘리시움   B1  NaN    04   
3      19417          히얼유고  Queen's Square   3F  NaN    93   
4      14468            더솔              기타  NaN  NaN   NaN   
..       ...           ...             ...  ...  ...   ...   
485    11691            이룸           Theot   1F    F    01   
486    14509          마징가디            제일평화   3F  NaN   198   
487     7455            에찌             청평화   5F    가    37   
488    11772           슬로우             동평화   1F    가  97-1   
489    18840            이음             APM   1F  NaN    49   

                         연락처            모바일        정산_전번  영업중  \
0                        NaN    01047579277          NaN  영업중   
1                        NaN  010-5470-8144          NaN  영업중   
2                01028435578    01028435578  01028435578  영업중   
3                 0269707771    01036977771          NaN  영업중   
4    01026621006,01092602613    01036322253  01081090084  영업중   
..                       ...            ...          ...  ...   
485             02-2117-8740  010-9956-8272          NaN  영업중   
486                      NaN    01058191891  01058191891  영업중   
487             02-2233-8346  010-2054-8346          NaN  영업중   
488             02-2117-8987    01020538987          NaN  영업중   
489              01074267076    01074267076  01074267076  영업중   

                           가입일자                        날짜      셀러ID  \
0                           NaN 2022-11-17 00:00:00+09:00  152073.0   
1    2022-05-17 09:14:22.220868 2022-11-17 00:00:00+09:00   25246.0   
2    2022-05-25 01:00:16.636045 2022-11-17 00:00:00+09:00   79289.0   
3    2022-08-25 09:38:13.361105 2022-11-17 00:00:00+09:00  164630.0   
4           2022-05-08 00:00:00 2022-11-17 00:00:00+09:00   83032.0   
..                          ...                       ...       ...   
485  2022-06-16 02:46:43.717577 2022-11-17 00:00:00+09:00   25608.0   
486  2022-10-05 10:27:49.010116 2022-11-17 00:00:00+09:00   83475.0   
487  2022-05-26 04:28:07.101928 2022-11-17 00:00:00+09:00   43818.0   
488  2022-10-06 13:40:25.143110 2022-11-17 00:00:00+09:00   74659.0   
489         2022-05-08 00:00:00 2022-11-17 00:00:00+09:00  152579.0   

           브랜드명_y             상가명 층_y   열_y     호 브랜드상태  
0          러브유어셀프     ddp패션몰(유어스)  1F  None    20   운영중  
1    더블엠(DOUBLEM)           Theot  1F     E    13   운영중  
2       블랙핑크_유아동복          혜양엘리시움  B1          04   운영중  
3            히얼유고  Queen's Square  3F  None    93   운영중  
4              더솔              기타      None  None   운영중  
..            ...             ...  ..   ...   ...   ...  
485            이룸           Theot  1F     F    01   운영중  
486          마징가디            제일평화  3F  None   198   운영중  
487            에찌             청평화  5F     가    37   운영중  
488           슬로우             동평화  1F     가  97-1   운영중  
489            이음             APM  1F  None    49   운영중  

[490 rows x 19 columns]

In [36]:
set(test_2.loc[test_2['브랜드상태'] == '폐업', 'brandId']).difference(set(test_2['brandId']).difference(test['brandId']))

set()

## 커플링패스 바이어

In [12]:
class ETLCouplingPassBuyer(DataExtractor):
    def __init__(self, host_ip, DBname, port, username, password, email_id, email_pw, from_date, to_date, client):
        super().__init__(host_ip, DBname, port, username, password, email_id, email_pw, client)
        self.from_date = from_date
        self.from_date_seoul = from_date + ' +0900'
        self.to_date = to_date
        self.to_date_seoul = to_date + ' +0900'

    def load_table(self, demo_test=False):
        self.NOSQL = {'dm_orderseries': {'SQL': {}},
                      'dm_buyer': {'SQL': {}}}
        # 데이터 필터링
        self.coupling = {
            'SQL': "select \"id\" as discount_promotion_id " + (
                "from tb_discount_promotion ") + (
                       "where ") + (
                       "\"title\" like '%커플링%' AND ") + (
                       f"\"end_at\" >= '{self.from_date_seoul}' AND ") + (
                       f"\"start_at\" < '{self.to_date_seoul}'"),
            'columns': ['discount_promotion_id', ]
        }
        super().load_table()

    def run(self):
        def type_extractor(x, types, recursive=None):
            try:
                if recursive is None:
                    return x[0][types]
                else:
                    return type_extractor([x[0][types]],
                                          types=recursive)
            except:
                return 'NaN'

        if not hasattr(self, 'dataframe_dm_buyer'):
            self.load_table()

        dataframe_orderseries = self.dataframe_dm_orderseries.copy()
        dataframe_buyer = self.dataframe_dm_buyer.copy()
        # 데이터 필터링
        dataframe_orderseries['createdAt'] = dataframe_orderseries[
            'createdAt'].dt.tz_localize('UTC').dt.tz_convert('Asia/Seoul')
        dataframe_orderseries = dataframe_orderseries[
            (dataframe_orderseries['createdAt'] >= self.from_date) & (
                    dataframe_orderseries['createdAt'] < self.to_date)]
        # 주문 데이터 필터링
        dataframe_orderseries = dataframe_orderseries[
            dataframe_orderseries['status_order'].apply(
                lambda x: x in [100, 200, 201, 202, 400])].reset_index(drop=True)
        dataframe_orderseries = dataframe_orderseries[
            dataframe_orderseries['productId'].apply(
                lambda x: len(str(x).split('_')) < 2)].reset_index(drop=True)
        # 바이어 필터링 
        discount_id = self.dataframe_coupling['discount_promotion_id'].values.tolist()
        coupling_ind = dataframe_buyer['discount_promotion_id'].dropna().apply(
            lambda x: np.any(list(map(lambda y: y in discount_id, x))))
        coupling_ind = coupling_ind[coupling_ind].index.values
        dm_account = pd.merge(dataframe_orderseries,
                              dataframe_buyer.loc[coupling_ind, 'buyerId'],
                              on='buyerId',
                              how='inner')
        # 주문 데이터 필터링(계속)
        dm_account = dm_account[(dm_account['initQualityInspectionFee'] == 0) & (
                dm_account['shippingCost'] == 0)]
        # 배송 관련 데이터 정리
        shipping_cost = dm_account[['orderId',
                                    'shippingCost',
                                    'dcshippingCost',
                                    'tracking_number']].drop_duplicates(['orderId',
                                                                         'shippingCost',
                                                                         'dcshippingCost']).reset_index(drop=True)
        # 상품 주문번호를 주문번호 단위로 groupby
        dm_account = dm_account.groupby(['orderId',
                                         'buyerId',
                                         'createdAt'])[['initTotalKRW',
                                                        'initSubtotal',
                                                        'initTax',
                                                        'finalSubtotal',
                                                        'finalTax',
                                                        'finalTotalKRW',
                                                        'initHandlingFee',
                                                        'dcHandlingFee',
                                                        'initQualityInspectionFee',
                                                        'dcQualityInspectionFee',
                                                        'initQuantity',
                                                        'finalQuantity']].sum().reset_index()
        # 배송 관련 데이터 정리(계속)
        dm_account = pd.merge(dm_account,
                              shipping_cost,
                              on='orderId',
                              how='left')
        dm_account['shipment_num_total'] = dm_account['tracking_number'].apply(
            lambda x: len(x) if not isinstance(x, float) else 0)
        dm_account['shipment_num_unique'] = dm_account['tracking_number'].apply(
            lambda x: len(set(x)) if not isinstance(x, float) else 0)
        dm_account = dm_account.drop('tracking_number', axis=1)
        dm_account = pd.merge(dm_account,
                              dataframe_buyer[['buyerId', 'bizName', 'orderName']],
                              on='buyerId',
                              how='left')
        dm_account.columns = ['order_id',
                              'buyer_id',
                              '주문일',
                              '최초 결제금액',
                              '상품금액',
                              'VAT',
                              '상품금액(최종)',
                              'VAT(최종)',
                              '최종 결제금액',
                              '사입 수수료',
                              '사입 수수료 할인',
                              '검수비',
                              '검수비 할인금액',
                              '주문수량',
                              '최종입고 수량',
                              '배송비',
                              '배송비 할인금액',
                              '오더별 택배발송 개수',
                              '오더별 택배발송 개수(오더별중복제거)',
                              '상호명',
                              '오더명']
        dm_account = dm_account[['order_id',
                                 '주문일',
                                 'buyer_id',
                                 '상호명',
                                 '오더명',
                                 '최초 결제금액',
                                 '상품금액',
                                 'VAT',
                                 '상품금액(최종)',
                                 'VAT(최종)',
                                 '최종 결제금액',
                                 '사입 수수료',
                                 '사입 수수료 할인',
                                 '배송비',
                                 '배송비 할인금액',
                                 '검수비',
                                 '검수비 할인금액',
                                 '주문수량',
                                 '최종입고 수량',
                                 '오더별 택배발송 개수(오더별중복제거)',
                                 '오더별 택배발송 개수']]

        dm_account['주문일'] = dm_account['주문일'].astype('str')
        self.dm_account = dm_account.copy()

In [13]:
couplingpassbuyer = ETLCouplingPassBuyer('restore.cy1gcw3mvrbf.ap-northeast-2.rds.amazonaws.com',
                              'linkshops',
                              '5432',
                              'linkshops',
                              '$dpdlvmflf4',
                              'ji.kwon@linkshops.com','cjswp25*',
                              '2023-01-01', '2023-02-01',
                                client)

In [14]:
couplingpassbuyer.run()

Table : coupling Load Complete
NoSQLTable : dm_orderseries Load Complete
NoSQLTable : dm_buyer Load Complete


In [40]:
couplingpassbuyer.dm_account.to_excel('회계_커플링패스바이어_20221017.xlsx', index = False)

In [74]:
test_id = np.random.choice(couplingpassbuyer.dm_account['order_id'], 10)

In [90]:
test_id

array([1382337, 1378641, 1390339, 1382372, 1389969, 1391516, 1388501,
       1380471, 1384046, 1384222], dtype=int64)

In [91]:
couplingpassbuyer.dm_account[couplingpassbuyer.dm_account['order_id'] == test_id[9]].T

484
order_id                            1384222
주문일        2022-09-15 10:53:33.856000+09:00
buyer_id                              59387
최초 결제금액                             60820.0
상품금액                                53500.0
VAT                                  5350.0
상품금액(최종)                            53500.0
VAT(최종)                              5350.0
최종 결제금액                             60820.0
사입 수수료                               1970.0
사입 수수료 할인                               0.0
배송비                                     0.0
배송비 할인금액                             7920.0
검수비                                     0.0
검수비 할인금액                             5100.0
주문수량                                   17.0
최종입고 수량                                17.0

## 커플링패스리스트

In [21]:
class ETLCouplingPassList(DataExtractor):
    def __init__(self, host_ip, DBname, port, username, password, email_id, email_pw, from_date, to_date, client):
        super().__init__(host_ip, DBname, port, username, password, email_id, email_pw, client)
        self.from_date = from_date
        self.to_date = to_date

    def load_table(self, demo_test=False):
        # 데이터 필터링
        self.coupling_pass = {'SQL': "select distinct(value.buyerId)," + (
                              "ids.description, ") + (
                              "ids.discount_promotion_id, ") + (
                              "value.created_at, ") + (
                              "ids.start_at, ") + (
                              "ids.end_at ") + (
                              "from(") + (
                "select \"id\" as discount_promotion_id, ") + (
                       "\"description\",") + (
                       "\"start_at\",") + (
                       "\"end_at\" ") + (
                       "from tb_discount_promotion ") + (
                       "where ") + (
                       # "\"end_at\" > '2022-06-01' and ") + (
                       "\"title\" like '%커플링%') ids ") + (
                       "join(") + (
                       "select \"buyer_id\" as buyerId") + (
                       ", \"discount_promotion_id\"") + (
                       ", \"use_yn\"") + (
                       ", \"created_at\"") + (
                       "from tb_discount_promotion_target_buyer ") + (
                       "where \"use_yn\" = 'Y') value ") + (
                       "on ids.\"discount_promotion_id\"=value.\"discount_promotion_id\""),
            'columns': ['buyerId',
                        'description',
                        'discount_promotion_id',
                        'createdAt',
                        'startAt',
                        'endAt']}
        self.NOSQL = {'dm_buyer': {'SQL': {}}}
        
        super().load_table()

    def run(self, threshold_membership = 1):
        def type_extractor(x, types, recursive=None):
            try:
                if recursive is None:
                    return x[0][types]
                else:
                    return type_extractor([x[0][types]],
                                          types=recursive)
            except:
                return 'NaN'
            
        if not hasattr(self, 'dataframe_dm_buyer'):
            self.load_table()
        dataframe_coupling = self.dataframe_coupling_pass.copy()
        dataframe_buyer = self.dataframe_dm_buyer.copy()      
        # 시간 기준 데이터 필터링
        dataframe_coupling['createdAt'] = dataframe_coupling['createdAt'].dt.tz_convert('Asia/Seoul')
        dataframe_coupling['startAt'] = dataframe_coupling['startAt'].dt.tz_convert('Asia/Seoul')
        dataframe_coupling['endAt'] = dataframe_coupling['endAt'].dt.tz_convert('Asia/Seoul')

        coupling_list = dataframe_coupling[(dataframe_coupling['endAt'] >= self.from_date) & (
                    dataframe_coupling['startAt'] < self.to_date)].drop_duplicates(
                    'buyerId').reset_index(drop = True)
        # dm_buyer와 Merge
        coupling_list = pd.merge(coupling_list[['buyerId','description']],
                        dataframe_buyer[['buyerId','name','bizName', 'roles']],
                         on = 'buyerId',
                        how = 'left')
        self.dm_account = coupling_list.copy()

In [25]:
couplingpassList = ETLCouplingPassList('restore.cy1gcw3mvrbf.ap-northeast-2.rds.amazonaws.com',
                              'linkshops',
                              '5432',
                              'linkshops',
                              '$dpdlvmflf4',
                              'ji.kwon@linkshops.com','cjswp25*',
                              '2022-10-01', '2022-11-01',
                                client)

In [26]:
couplingpassList.run()

Table : coupling_pass Load Complete
NoSQLTable : dm_buyer Load Complete


## 수수료 0% 이벤트

In [4]:
class ETLCZeroRatingEvent(DataExtractor):
    def __init__(self, host_ip, DBname, port, username, password, email_id, email_pw, from_date, to_date, client):
        super().__init__(host_ip, DBname, port, username, password, email_id, email_pw, client)
        self.from_date = from_date
        self.to_date = to_date

    def load_table(self, demo_test=False):
        self.NOSQL = {'dm_orderseries': {'SQL': {}}}
        
        super().load_table()

    def run(self, threshold_membership = 1):
        def type_extractor(x, types, recursive=None):
            try:
                if recursive is None:
                    return x[0][types]
                else:
                    return type_extractor([x[0][types]],
                                          types=recursive)
            except:
                return 'NaN'
            
        if not hasattr(self, 'dataframe_dm_orderseries'):
            self.load_table()
        
        dataframe_orderseries = self.dataframe_dm_orderseries.copy()
    
        dataframe_orderseries['createdAt'] = \
        dataframe_orderseries['createdAt'].dt.tz_localize('UTC').dt.tz_convert('Asia/Seoul')
        # 데이터 필터링
        dm_account = dataframe_orderseries[(
                 dataframe_orderseries['createdAt'] >= self.from_date) & (
                 dataframe_orderseries['createdAt'] < self.to_date)]
        dm_account = dm_account.reset_index(drop = True)
        dm_account['month'] = \
                dm_account['createdAt'].dt.year.astype('str') + \
                                                          '-' + \
                dm_account['createdAt'].dt.month.astype('str')
        dm_account = dm_account[['buyerId',
                         'orderId',
                         'month',
                         'productId',
                         'countryCode',
                         'status_order',
                         'initTotalKRW',
                         'initSubtotal',
                         'initHandlingFee',
                         'dcHandlingFee',
                         'initQualityInspectionFee',
                         'shippingCost',
                         'finalShippingCost'
                         ]]
        dm_account = pd.merge(dm_account,
                     Series([100,200,201,202], name = 'status_order'),
                     on = 'status_order')
        not_sero = dm_account['productId'].apply( lambda x : len(str(x).split('_')) < 2)
        dm_account = dm_account[not_sero].reset_index(drop = True)
        # 0% 이벤트 대상 내역 추출
        ids = dm_account[dm_account['initHandlingFee'] == 0]['buyerId'].unique()
        # 배송비 처리
        dm_account = pd.merge(dm_account,
                              Series(ids, name = 'buyerId'),
                              on = 'buyerId')
        shipping_cost = dm_account[['buyerId',
                                    'orderId',
                                    'shippingCost',
                                    'finalShippingCost']].drop_duplicates()
        # 상품주문번호 단위를 주문번호 단위로 통합(groupby sum)
        dm_account = dm_account.groupby(['buyerId', 'orderId', 'countryCode', 'month'])[                                              
                                             ['initTotalKRW',
                                             'initSubtotal',
                                             'initHandlingFee',
                                             'dcHandlingFee',
                                             'initQualityInspectionFee'         
                                             ]].sum().reset_index()
        # 배송비 처리(계속)
        dm_account = pd.merge(dm_account, shipping_cost, on = ['buyerId','orderId'])
        # buyerId 단위로 통합(groupby Sum)
        dm_account = dm_account.groupby(['buyerId','countryCode','month'])[
                                                        ['initTotalKRW',
                                                         'initSubtotal',
                                                         'initHandlingFee',
                                                         'dcHandlingFee',
                                                         'initQualityInspectionFee',
                                                         'shippingCost'
                                                         ]].sum().reset_index()
        dm_account.columns = ['buyerid',
                              '국가코드',
                              '년-월',
                              '결제금액',
                              '상품금액',
                              '판매수수료',
                              '판매수수료 할인금액',
                              '검수검품비',
                              '배송비']
        dm_account = dm_account[['년-월',
                                 'buyerid',
                                 '결제금액',
                                 '상품금액',
                                 '판매수수료',
                                 '판매수수료 할인금액',
                                 '검수검품비',
                                 '배송비',
                                 '국가코드']]
        dm_account['결제금액'] += dm_account['배송비']
        dm_account.loc[dm_account['국가코드'] != 'KR', '상품금액'] += \
        dm_account.loc[dm_account['국가코드'] != 'KR', '상품금액'] * 0.1
        
        self.dm_account = dm_account.copy()

In [5]:
zeroratingevent = ETLCZeroRatingEvent('restore.cy1gcw3mvrbf.ap-northeast-2.rds.amazonaws.com',
                              'linkshops',
                              '5432',
                              'linkshops',
                              '$dpdlvmflf4',
                              'ji.kwon@linkshops.com','cjswp25*',
                              '2022-09-01', '2022-10-01',
                                client)

In [10]:
zeroratingevent.run()

NoSQLTable : dm_orderseries Load Complete


In [64]:
zeroratingevent.dm_account

년-월  buyerid      결제금액      상품금액    판매수수료  판매수수료 할인금액   검수검품비  \
0    2022-9     1234  924940.0  831000.0  10840.0         0.0     0.0   
1    2022-9     8615  149700.0  117700.0      0.0      9420.0  2600.0   
2    2022-9    11047  325320.0  290000.0   6320.0         0.0     0.0   
3    2022-9    11215  594230.0  524500.0  17280.0         0.0     0.0   
4    2022-9    11575  294880.0  262000.0   6680.0         0.0     0.0   
..      ...      ...       ...       ...      ...         ...     ...   
233  2022-9   165181  768150.0  683800.0  15970.0         0.0     0.0   
234  2022-9   165274  290400.0  264000.0      0.0         0.0     0.0   
235  2022-9   165375  639060.0  577500.0   3810.0         0.0     0.0   
236  2022-9   165396  611440.0  547000.0   9740.0         0.0     0.0   
237  2022-9   165432  148500.0  135000.0      0.0         0.0     0.0   

         배송비 국가코드  
0        0.0   KR  
1    29400.0   TW  
2        0.0   KR  
3        0.0   KR  
4        0.0   KR  
..       ...  ...  
233      0.0   KR  
234      0.0   KR  
235      0.0   KR  
236      0.0   KR  
237      0.0   KR  

[238 rows x 9 columns]

## 블랙라벨세일즈리포트

In [121]:
class ETLBlackLabelReport(DataExtractor):
    def __init__(self, host_ip, DBname, port, username, password, email_id, email_pw, prev_date, current_date, client):
        super().__init__(host_ip, DBname, port, username, password, email_id, email_pw, client)
        self.prev_from_date = prev_date[0]
        self.prev_to_date = prev_date[1]
        self.current_from_date = current_date[0]
        self.current_to_date = current_date[1]

    def load_table(self, demo_test=False):
        self.NOSQL = {'dm_orderseries': {'SQL': {}},
                      'dm_seller': {'SQL': {}},
                      'dm_merchandise': {'SQL': [{}, {'productId': 1, 'SKUdata.name': 1}]}}
        # 브랜드별 / 연월별 GroupBy
        self.wishes = {"SQL": "select wis.\"brandId\"" + (
            ", to_char(wis.\"createdAt\" at time zone 'Asia/Seoul', 'YYYY-MM') as yearmon") + (
                                  ", count(*) from wishes wis where ") + (
                                  "wis.status = 100 ") + (
                                  f"and wis.\"createdAt\" between '{self.prev_from_date + ' +0900'}' and ") + (
                                  f"'{self.current_to_date + ' +0900'}'") + (
                                  " GROUP BY wis.\"brandId\", to_char(wis.\"createdAt\" at time zone 'Asia/Seoul', 'YYYY-MM')"),
                       "columns": ['brandId', 'yearmon', 'count']}
        # 누적 총 개수 도출
        self.wishes_total = {"SQL": "select wis.\"brandId\"" + (
            ", count(*) from wishes wis where ") + (
                                        "wis.status = 100 ") + (
                                        " GROUP BY wis.\"brandId\""),
                             "columns": ['brandId', '누적 위시리스트 추가 횟수']}
        # 브랜드별 / 연월별 GroupBy
        self.brand_favorite = {"SQL": "select tfb.\"brand\"" + (
            ", to_char(tfb.\"created_at\" at time zone 'Asia/Seoul', 'YYYY-MM') as yearmon") + (
                                          ", count(*) from tb_favorite_brand tfb where ") + (
                                          f"tfb.\"created_at\" between '{self.prev_from_date + ' +0900'}' and ") + (
                                          f"'{self.current_to_date + ' +0900'}'") + (
                                          " GROUP BY tfb.\"brand\", to_char(tfb.\"created_at\" at time zone 'Asia/Seoul', 'YYYY-MM')"),
                               "columns": ['brandId', 'yearmon', 'count']}
        # 누적 총 개수 도출
        self.brand_total = brand_total = {"SQL": "select tfb.\"brand\"" + (
            ", count(*) from tb_favorite_brand tfb") + (
                                                     " GROUP BY tfb.\"brand\""),
                                          "columns": ['brandId', '누적 단골 바이어 등록 수']}

        super().load_table()

        current_dir = os.path.dirname(os.path.abspath(__file__))
        blackalbel_brand = pd.read_csv(f'{current_dir}/init_data/blacklabel_brand.csv')
        self.blackalbel_brand = tuple(blackalbel_brand['brandId'].values)

    def run(self, threshold_membership=1):
        def type_extractor(x, types, recursive=None):
            try:
                if recursive is None:
                    return x[0][types]
                else:
                    return type_extractor([x[0][types]],
                                          types=recursive)
            except:
                return 'NaN'

        def extract_best_country(x, top=3):
            finaltotal = x.groupby('countryCode')['finalSubtotal'].sum()
            ranking = finaltotal.sort_values(
                ascending=False).iloc[0:top].reset_index()
            table = DataFrame()
            for i, j in ranking.iterrows():
                j.index = [f'출고액 {i + 1}위 국가',
                           f'{i + 1}위 국가 출고액 비중']
                j = DataFrame(j).T
                j.index = [0]
                table = pd.concat([table, j], axis=1)
            return table

        def extract_best_product(x, top=3, how='cnt'):
            if how == 'cnt':
                finaltotal = x.groupby(['productName'])['productName'].count()
            if how == 'quantity':
                finaltotal = x.groupby(['productName'])['initQuantity'].sum()
            finaltotal.name = 'count'
            ranking = finaltotal.sort_values(
                ascending=False).iloc[0:top].reset_index()
            table = DataFrame()
            for i, j in ranking.iterrows():
                j.index = [f'판매량 {i + 1}위 상품명',
                           f'판매량 {i + 1}위 상품 판매 수량']
                j = DataFrame(j).T
                j.index = [0]
                table = pd.concat([table, j], axis=1)
            return table

        if not hasattr(self, 'dataframe_brand_favorite'):
            self.load_table()

        start_time = time.time()

        dataframe_merchandise = self.dataframe_dm_merchandise.copy()
        dataframe_orderseries = self.dataframe_dm_orderseries.copy()
        dataframe_seller = self.dataframe_dm_seller.copy()
        dataframe_wishes = self.dataframe_wishes.copy()
        dataframe_wishes_total = self.dataframe_wishes_total.copy()
        dataframe_brand_favor = self.dataframe_brand_favorite.copy()
        dataframe_brand_total = self.dataframe_brand_total.copy()
        blacklabel_brand = self.blackalbel_brand
        # 기본 데이터 준비
        dataframe_merchandise = dataframe_merchandise[['productId', 'SKUdata']].drop_duplicates(['productId'])
        dataframe_orderseries['createdAt'] = dataframe_orderseries[
            'createdAt'].dt.tz_localize('UTC').dt.tz_convert('Asia/Seoul')
        dataframe_orderseries = pd.merge(dataframe_seller[['brandId', 'sellerId']],
                                         dataframe_orderseries,
                                         on='brandId',
                                         how='right')
        dm_account = pd.merge(dataframe_orderseries,
                              Series(list(set(dataframe_orderseries['status_order_prod'].unique()).difference([0, -1])),
                                     name='status_order_prod'),
                              on='status_order_prod')
        dm_account = dm_account[dm_account['initKRW'] > 0]
        # 결측값 처리
        dm_account.loc[dm_account['initSubtotal'].isna(), 'initSubtotal'] = \
            dm_account.loc[dm_account['initSubtotal'].isna(), 'initTotalKRW']
        dm_account.loc[dm_account['finalSubtotal'].isna(), 'finalSubtotal'] = \
            dm_account.loc[dm_account['finalSubtotal'].isna(), 'finalTotalKRW']
        dataframe_merchandise['productName'] = dataframe_merchandise['SKUdata'].apply(
            lambda x: type_extractor([x], 'name', recursive='ko'))
        # 기본 데이터 준비(계속)
        dm_account = pd.merge(dm_account,
                              dataframe_merchandise[['productId', 'productName']],
                              on='productId',
                              how='left')
        # 블랙라벨 브랜드 필터링
        dm_account = pd.merge(dm_account,
                              Series(blacklabel_brand, name='brandId'),
                              on='brandId')
        # 전월 실적 처리
        prev_month = dm_account[(dm_account['createdAt'] >= self.prev_from_date) & (
                dm_account['createdAt'] < self.prev_to_date)]
        prev_month_groupby = prev_month.groupby('sellerId')[['initQuantity',
                                                             'finalQuantity',
                                                             'initSubtotal',
                                                             'finalSubtotal']].sum().reset_index()
        prev_month_groupby.columns = ['sellerId',
                                      '전월 주문 수량',
                                      '전월 출고수량',
                                      '전월 주문액',
                                      '전월 출고액']
        # 당월 실적 처리
        current_month = dm_account[(dm_account['createdAt'] >= self.current_from_date) & (
                dm_account['createdAt'] < self.current_to_date)]
        current_month_groupby = current_month.groupby('sellerId')[['initQuantity',
                                                                   'finalQuantity',
                                                                   'initSubtotal',
                                                                   'finalSubtotal']].sum().reset_index()
        current_month_groupby.columns = ['sellerId',
                                         '당월 주문 수량',
                                         '당월 출고수량',
                                         '당월 주문액',
                                         '당월 출고액']
        # current_month_groupby / prev_month_groupby merge
        dm_account = pd.merge(prev_month_groupby,
                              current_month_groupby,
                              on='sellerId',
                              how='outer').fillna(0)
        dm_account = pd.merge(dm_account,
                              dataframe_seller[['sellerId', 'name']].drop_duplicates(subset = 'sellerId',
                                                                                     keep = 'last'),
                              on='sellerId',
                              how='left')
        # 셀러별 Top N 국가 도출
        country_ranking = current_month.groupby('sellerId').apply(
            lambda x: extract_best_country(x))
        dm_account = pd.merge(dm_account,
                              country_ranking.reset_index().drop('level_1', axis=1),
                              on='sellerId',
                              how='left')
        # 셀러별 Top N 상품 도출
        product_ranking = current_month.groupby('sellerId').apply(lambda x: extract_best_product(x))
        product_ranking = product_ranking.reset_index().drop('level_1', axis=1)
        # Top N 국가 / 상품 Merge
        dm_account = pd.merge(dm_account,
                              product_ranking,
                              on='sellerId',
                              how='left')
        # wishes 처리 / wishes_total 처리 / brand_favor 처리 / brand_total 처리
        dataframe_wishes = pd.merge(dataframe_wishes, Series(blacklabel_brand, name='brandId'),
                                    on='brandId')
        dataframe_wishes_total = pd.merge(dataframe_wishes_total, Series(blacklabel_brand, name='brandId'),
                                          on='brandId')
        dataframe_brand_favor = pd.merge(dataframe_brand_favor, Series(blacklabel_brand, name='brandId'),
                                         on='brandId')
        dataframe_brand_total = pd.merge(dataframe_brand_total, Series(blacklabel_brand, name='brandId'),
                                         on='brandId')

        dataframe_wishes = pd.merge(dataframe_seller[['sellerId', 'brandId']], dataframe_wishes,
                                    on='brandId',
                                    how='right')
        dataframe_wishes_total = pd.merge(dataframe_seller[['sellerId', 'brandId']], dataframe_wishes_total,
                                          on='brandId',
                                          how='right')
        dataframe_brand_favor = pd.merge(dataframe_seller[['sellerId', 'brandId']], dataframe_brand_favor,
                                         on='brandId',
                                         how='right')
        dataframe_brand_total = pd.merge(dataframe_seller[['sellerId', 'brandId']], dataframe_brand_total,
                                         on='brandId',
                                         how='right')

        dataframe_wishes = dataframe_wishes.groupby(['sellerId', 'yearmon'])['count'].sum().reset_index()
        dataframe_brand_favor = dataframe_brand_favor.groupby(['sellerId', 'yearmon'])['count'].sum().reset_index()
        dataframe_wishes_total = dataframe_wishes_total.groupby('sellerId')['누적 위시리스트 추가 횟수'].sum().reset_index()

        dataframe_brand_total = dataframe_brand_total.groupby('sellerId')['누적 단골 바이어 등록 수'].sum().reset_index()
        #dataframe_wishes / dataframe_brand_favor 피보팅
        result = dataframe_wishes.pivot(columns=['yearmon'], values=['count'], index=['sellerId'])
        result = result.reset_index()
        result.columns = ['sellerId', '전월 위시리스트 추가 횟수', '당월 위시리스트 추가 횟수']
        result_wishes = result.fillna(0)
        result = dataframe_brand_favor.pivot(columns=['yearmon'], values=['count'], index=['sellerId'])
        result = result.reset_index()
        result.columns = ['sellerId', '전월 단골 바이어 등록 수', '당월 단골 바이어 등록 수']
        result_brand = result.fillna(0)
        #temp와 dataframe_wishes_total, dataframe_brand_total MERGE
        temp = pd.merge(result_wishes, result_brand, on='sellerId', how='outer').fillna(0)
        result = pd.merge(dataframe_wishes_total, temp, on='sellerId', how='outer').fillna(0)
        result = pd.merge(dataframe_brand_total, result, on='sellerId', how='outer').fillna(0)
        # Dataframe_wishes, dataframe_brand_favor MERGE
        dm_account = pd.merge(dm_account, result, on='sellerId', how='left')
        dm_account = dm_account.rename({'name': '브랜드명'}, axis=1)
        # 한글명 컬럼으로 변경
        dm_account['집계기간'] = self.current_from_date.split('-')[0] + '년 ' + \
                             self.current_from_date.split('-')[1] + '월'
        # 공란 처리 컬럼 추가
        dm_account[['번역처리값',
                    '당월 총 큐레이션 제공 횟수',
                    '전월 대비 당월 주문 금액',
                    '전월 대비 당월 주문액 변화(%)',
                    '전월 대비 당원 주문 수량 변화(건)',
                    '전월',
                    '당월',
                    '전월 대비 당월 출고액 변화(%)',
                    '전월 대비 당월 출고 수량 변화']] = np.nan
        dm_account = dm_account[["브랜드명",
                                 "sellerId",
                                 "집계기간",
                                 "번역처리값",
                                 "당월 총 큐레이션 제공 횟수",
                                 "전월 대비 당월 주문 금액",
                                 "전월 대비 당월 주문액 변화(%)",
                                 "전월 대비 당원 주문 수량 변화(건)",
                                 "전월 주문액",
                                 "전월 주문 수량",
                                 "전월 출고액",
                                 "전월 출고수량",
                                 "당월 주문액",
                                 "당월 주문 수량",
                                 "당월 출고액",
                                 "당월 출고수량",
                                 "전월",
                                 "당월",
                                 "전월 대비 당월 출고액 변화(%)",
                                 "전월 대비 당월 출고 수량 변화",
                                 "전월 위시리스트 추가 횟수",
                                 "당월 위시리스트 추가 횟수",
                                 "누적 위시리스트 추가 횟수",
                                 "전월 단골 바이어 등록 수",
                                 "당월 단골 바이어 등록 수",
                                 "누적 단골 바이어 등록 수",
                                 "출고액 1위 국가",
                                 "1위 국가 출고액 비중",
                                 "출고액 2위 국가",
                                 "2위 국가 출고액 비중",
                                 "출고액 3위 국가",
                                 "3위 국가 출고액 비중",
                                 "판매량 1위 상품명",
                                 "판매량 1위 상품 판매 수량",
                                 "판매량 2위 상품명",
                                 "판매량 2위 상품 판매 수량",
                                 "판매량 3위 상품명",
                                 "판매량 3위 상품 판매 수량"]]
        self.dm_account = dm_account.copy()

In [123]:
blacklabelreport = ETLBlackLabelReport('restore.cy1gcw3mvrbf.ap-northeast-2.rds.amazonaws.com',
                              'linkshops',
                              '5432',
                              'linkshops',
                              '$dpdlvmflf4',
                              'ji.kwon@linkshops.com','cjswp25*',
                              ['2022-11-01 00:00:00','2022-12-01 00:00:00'], 
                              ['2022-12-01 00:00:00','2023-01-01 00:00:00'],
                                client)

In [124]:
__file__ = 'C:/Users/Administrator/Documents/data_extraction_advanced/test.py'

In [125]:
blacklabelreport.run()

Table : wishes Load Complete
Table : brand_total Load Complete
Table : brand_favorite Load Complete
Table : wishes_total Load Complete
NoSQLTable : dm_orderseries Load Complete
NoSQLTable : dm_seller Load Complete
NoSQLTable : dm_merchandise Load Complete


In [130]:
test = pd.merge(blacklabelreport.dm_account,
         pd.read_excel('회계_블랙라벨세일즈리포트_2022-12-01 00_00_00_2023-01-01 00_00_00.xlsx'),
         on = ['sellerId'])

In [203]:
blacklabelreport.dm_account.T

0                         1          2   \
브랜드명                      유니온서울                      레드이어        미스홍   
sellerId                 2187.0                    2283.0     3316.0   
집계기간                  2022년 12월                 2022년 12월  2022년 12월   
번역처리값                       NaN                       NaN        NaN   
당월 총 큐레이션 제공 횟수             NaN                       NaN        NaN   
전월 대비 당월 주문 금액              NaN                       NaN        NaN   
전월 대비 당월 주문액 변화(%)          NaN                       NaN        NaN   
전월 대비 당원 주문 수량 변화(건)        NaN                       NaN        NaN   
전월 주문액                 299000.0                  792000.0  1647000.0   
전월 주문 수량                   15.0                      30.0      164.0   
전월 출고액                 299000.0                  676000.0  1235000.0   
전월 출고수량                    15.0                      25.0      114.0   
당월 주문액                      0.0                  652000.0  1113000.0   
당월 주문 수량                    0.0                      23.0       92.0   
당월 출고액                      0.0                  627000.0   935000.0   
당월 출고수량                     0.0                      22.0       74.0   
전월                          NaN                       NaN        NaN   
당월                          NaN                       NaN        NaN   
전월 대비 당월 출고액 변화(%)          NaN                       NaN        NaN   
전월 대비 당월 출고 수량 변화           NaN                       NaN        NaN   
전월 위시리스트 추가 횟수              0.0                      92.0      111.0   
당월 위시리스트 추가 횟수              0.0                     102.0      142.0   
누적 위시리스트 추가 횟수              0.0                    4223.0     6905.0   
전월 단골 바이어 등록 수              9.0                      13.0       12.0   
당월 단골 바이어 등록 수              0.0                       9.0        7.0   
누적 단골 바이어 등록 수              428                       578        910   
출고액 1위 국가                   NaN                        KR         HK   
1위 국가 출고액 비중                NaN                  260000.0   628000.0   
출고액 2위 국가                   NaN                        HK         KR   
2위 국가 출고액 비중                NaN                  146000.0   238000.0   
출고액 3위 국가                   NaN                        CA         TW   
3위 국가 출고액 비중                NaN                  140000.0    69000.0   
판매량 1위 상품명                  NaN  1031 인디고 워시드 코지 스트레이트팬츠         말캉T   
판매량 1위 상품 판매 수량             NaN                         2          4   
판매량 2위 상품명                  NaN          3909 와이드 코듀로이 팬츠      챠밍가디건   
판매량 2위 상품 판매 수량             NaN                         2          4   
판매량 3위 상품명                  NaN      057 플리츠 코듀로이 데미지 워시드       보아JP   
판매량 3위 상품 판매 수량             NaN                         1          4   

                                        3          4                 5   \
브랜드명                                    아로      런_APM            제이_지하2   
sellerId                            4216.0     6149.0            6981.0   
집계기간                             2022년 12월  2022년 12월         2022년 12월   
번역처리값                                  NaN        NaN               NaN   
당월 총 큐레이션 제공 횟수                        NaN        NaN               NaN   
전월 대비 당월 주문 금액                         NaN        NaN               NaN   
전월 대비 당월 주문액 변화(%)                     NaN        NaN               NaN   
전월 대비 당원 주문 수량 변화(건)                   NaN        NaN               NaN   
전월 주문액                            175000.0   860000.0          871000.0   
전월 주문 수량                               7.0       13.0              42.0   
전월 출고액                            164000.0   832000.0          756000.0   
전월 출고수량                                6.0       12.0              36.0   
당월 주문액                            168000.0   148000.0           99000.0   
당월 주문 수량                               8.0        3.0               5.0   
당월 출고액                            109000.0   148000.0           69000.0   
당월 출고수량             

In [226]:
columns = \
['전월 주문액',
'전월 주문 수량',
'전월 출고액',
'전월 출고수량',
'당월 주문액',
'당월 주문 수량',
'당월 출고액',
'당월 출고수량',
'전월 위시리스트 추가 횟수',
'당월 위시리스트 추가 횟수',
'누적 위시리스트 추가 횟수',
'전월 단골 바이어 등록 수',
'당월 단골 바이어 등록 수',
'누적 단골 바이어 등록 수',
'출고액 1위 국가',
'출고액 2위 국가',
'출고액 3위 국가',
'1위 국가 출고액 비중',
'2위 국가 출고액 비중',
'3위 국가 출고액 비중',
'판매량 1위 상품명',
'판매량 2위 상품명',
'판매량 3위 상품명',
'판매량 1위 상품 판매 수량',
'판매량 2위 상품 판매 수량',
'판매량 3위 상품 판매 수량']

In [227]:
unit_test_result = dict()
for i in columns:
    if isinstance(test[f'{i}_x'], float):
        unit_test_result.update({i : np.all((test[f'{i}_x'].subtract(test[f'{i}_y'])) == 0)})
    else :
        unit_test_result.update({i : np.all((test[f'{i}_x'].dropna() == test[f'{i}_y'].dropna()))})

In [228]:
unit_test_result

{'전월 주문액': True,
 '전월 주문 수량': True,
 '전월 출고액': True,
 '전월 출고수량': True,
 '당월 주문액': True,
 '당월 주문 수량': True,
 '당월 출고액': False,
 '당월 출고수량': False,
 '전월 위시리스트 추가 횟수': False,
 '당월 위시리스트 추가 횟수': False,
 '누적 위시리스트 추가 횟수': False,
 '전월 단골 바이어 등록 수': False,
 '당월 단골 바이어 등록 수': False,
 '누적 단골 바이어 등록 수': False,
 '출고액 1위 국가': True,
 '출고액 2위 국가': True,
 '출고액 3위 국가': True,
 '1위 국가 출고액 비중': False,
 '2위 국가 출고액 비중': True,
 '3위 국가 출고액 비중': True,
 '판매량 1위 상품명': False,
 '판매량 2위 상품명': False,
 '판매량 3위 상품명': False,
 '판매량 1위 상품 판매 수량': True,
 '판매량 2위 상품 판매 수량': True,
 '판매량 3위 상품 판매 수량': True}

In [159]:
i = columns[6]

In [162]:
test[f'{i}_x'].subtract(test[f'{i}_y']).sort_values()

0         0.0
36        0.0
37        0.0
38        0.0
39        0.0
       ...   
70        0.0
27    26000.0
1     28000.0
46    58000.0
53    90000.0
Length: 71, dtype: float64

In [188]:
i = columns[7]

In [189]:
test[f'{i}_x'].subtract(test[f'{i}_y']).sort_values()

0     0.0
36    0.0
37    0.0
38    0.0
39    0.0
     ... 
70    0.0
46    1.0
27    1.0
1     1.0
53    5.0
Length: 71, dtype: float64

In [190]:
i = columns[8]

In [191]:
test[f'{i}_x'].subtract(test[f'{i}_y']).sort_values()

57   -29.0
6     -8.0
38    -5.0
41    -5.0
69    -4.0
      ... 
34     0.0
37     0.0
40     0.0
21     0.0
70     0.0
Length: 71, dtype: float64

In [192]:
i = columns[9]

In [193]:
test[f'{i}_x'].subtract(test[f'{i}_y']).sort_values()

57   -25.0
41   -14.0
46    -8.0
17    -8.0
38    -7.0
      ... 
36     0.0
37     0.0
18     0.0
70     0.0
59     1.0
Length: 71, dtype: float64

In [194]:
i = columns[10]

In [195]:
test[f'{i}_x'].subtract(test[f'{i}_y']).sort_values()

57    -86.0
52    -12.0
25    -12.0
4      -1.0
26     -1.0
      ...  
23     51.0
16     63.0
61     70.0
64     82.0
41    138.0
Length: 71, dtype: float64

In [196]:
i = columns[11]

In [197]:
test[f'{i}_x'].subtract(test[f'{i}_y']).sort_values()

48   -1.0
0     0.0
50    0.0
49    0.0
47    0.0
     ... 
21    0.0
20    0.0
19    0.0
25    0.0
70    0.0
Length: 71, dtype: float64

In [198]:
i = columns[12]

In [199]:
test[f'{i}_x'].subtract(test[f'{i}_y']).sort_values()

23   -1.0
0     0.0
50    0.0
49    0.0
48    0.0
     ... 
21    0.0
20    0.0
19    0.0
26    0.0
70    0.0
Length: 71, dtype: float64

In [200]:
i = columns[13]

In [201]:
test[f'{i}_x'].subtract(test[f'{i}_y']).sort_values()

15    -1
55    -1
0      0
30     0
31     0
      ..
52     8
46     8
61     9
64    11
6     12
Length: 71, dtype: int64

In [210]:
i = columns[14]

In [214]:
test.loc[test[f'{i}_x'] != test[f'{i}_y'], [f'{i}_x',f'{i}_y']]

출고액 1위 국가_x 출고액 1위 국가_y
0          NaN         NaN
9          NaN         NaN
49         NaN         NaN

In [215]:
i = columns[15]

In [216]:
test.loc[test[f'{i}_x'] != test[f'{i}_y'], [f'{i}_x',f'{i}_y']]

출고액 2위 국가_x 출고액 2위 국가_y
0          NaN         NaN
4          NaN         NaN
9          NaN         NaN
10         NaN         NaN
11         NaN         NaN
12         NaN         NaN
20         NaN         NaN
29         NaN         NaN
34         NaN         NaN
39         NaN         NaN
40         NaN         NaN
44         NaN         NaN
45         NaN         NaN
49         NaN         NaN
51         NaN         NaN
54         NaN         NaN
65         NaN         NaN
70         NaN         NaN

In [231]:
i = columns[20]

In [233]:
test.loc[test[f'{i}_x'] != test[f'{i}_y'], [f'{i}_x',f'{i}_y']]

판매량 1위 상품명_x 판매량 1위 상품명_y
0                    NaN          NaN
9                    NaN          NaN
49                   NaN          NaN
57       sale2023#)밀라노니트        밀라노니트
59  sale2023)포인트자수 니트 폴라    신상)터틀 맨투맨

In [238]:
i = columns[23]

In [239]:
test[f'{i}_x'].subtract(test[f'{i}_y']).sort_values()

1     0.0
38    0.0
39    0.0
40    0.0
41    0.0
     ... 
26    0.0
70    0.0
0     NaN
9     NaN
49    NaN
Length: 71, dtype: object

In [180]:
test.loc[27,f'{i}_y']

428000

In [176]:
test.iloc[[27]]

브랜드명  sellerId     집계기간_x  번역처리값_x  당월 총 큐레이션 제공 횟수_x  전월 대비 당월 주문 금액_x  \
27  브라보   50290.0  2022년 12월      NaN                NaN               NaN   

    전월 대비 당월 주문액 변화(%)_x  전월 대비 당원 주문 수량 변화(건)_x  전월 주문액_x  전월 주문 수량_x  ...  \
27                   NaN                     NaN  494000.0        22.0  ...   

    출고액 2위 국가_y  2위 국가 출고액 비중_y  출고액 3위 국가_y  3위 국가 출고액 비중_y   판매량 1위 상품명_y  \
27           TW         38000.0          NaN             NaN  몬타나 피그 후드(기모)   

    판매량 1위 상품 판매 수량_y  판매량 2위 상품명_y  판매량 2위 상품 판매 수량_y  판매량 3위 상품명_y  \
27                4.0      네트 자수 팬츠                2.0      네트 자수 후드   

    판매량 3위 상품 판매 수량_y  
27                2.0  

[1 rows x 75 columns]

In [168]:
blacklabelreport.dataframe_dm_seller[blacklabelreport.dataframe_dm_seller['sellerId'] == 50290]

_id  sellerId  status_seller  sellerName  \
6833  63bee6f2f4988ea224fe0afc   50290.0          100.0  브라보(BRAVO)   

                   createdAt               updatedAt  \
6833 2018-07-03 20:03:17.065 2023-01-11 12:38:38.017   

                           lastLoginAt  brandId name  status_brand  ...  \
6833  2023-01-11 12:38:38.017526+00:00  11548.0  브라보         100.0  ...   

      overshippedHistory  cumsumOvershipped  cusumOrderAmount  \
6833                 NaN                NaN         2130000.0   

     cusumPurchaseAmount cusumOrderQuantity cusumPurchaseQuantity uploadedSku  \
6833            815000.0              104.0                  36.0        29.0   

                                    waitshipmentHistory  avgWaitShipment  \
6833  [{'18358740': {'start_date': 2022-11-07 04:50:...         2.888889   

     avgWaitTerm  
6833         2.0  

[1 rows x 38 columns]

In [183]:
a = blacklabelreport.dataframe_dm_orderseries[blacklabelreport.dataframe_dm_orderseries['brandId'] == 11548]

In [181]:
a['initTotalKRW'].sum()

2544582.0

In [127]:
holidays = pd.read_csv('../데이터 추출/init_data/holiday.csv')

## 환불 누락의심건 도출

In [27]:
class ETLOmittedRefund(DataExtractor):
    def __init__(self, host_ip, DBname, port, username, password, email_id, email_pw, from_date, to_date, client):
        super().__init__(host_ip, DBname, port, username, password, email_id, email_pw, client)
        self.from_date = from_date
        self.to_date = to_date

    def load_table(self, demo_test=False):
        # Settlement ID 기반 데이터 필터링
        self.deposit = {'SQL' : "select " + (
                   "sop.\"orderId\",") + (
                   "sop.\"orderProductId\",") + (
                   "sop.\"buyerId\",") + (
                   "sop.\"brandId\",") + (
                   "td.\"status\",") + (
                   "td.\"created_at\"") + (
                   "from tb_deposit td ") + (
            # 필요 데이터 JOIN
                   "join settlement_bank_transfers sbt on td.additional_deposit_id = sbt.id ") + (
                   "join settlement_order_products sop on sop.\"settlementBankTransferId\" = td.additional_deposit_id ") + (
                   "where td.additional_deposit_type = 200 AND ") + (
                   f"td.\"created_at\" >= '{self.from_date}'"),
           'columns' : ['orderId',
                      'orderProductId',
                      'buyerId',
                      'brandId',
                      'status',
                      'createdAt']}
        self.NOSQL = {'dm_orderseries': {'SQL': {}}}
        
        super().load_table()

    def run(self, threshold_membership = 1):
        def type_extractor(x, types, recursive=None):
            try:
                if recursive is None:
                    return x[0][types]
                else:
                    return type_extractor([x[0][types]],
                                          types=recursive)
            except:
                return 'NaN'
            
        if not hasattr(self, 'dataframe_dm_orderseries'):
            self.load_table()
            
        dataframe_orderseries = self.dataframe_dm_orderseries.copy()
        dataframe_deposit = self.dataframe_deposit.copy()
        # 시간 기준 데이터 필터링
        dataframe_orderseries = \
        dataframe_orderseries[
            (dataframe_orderseries['createdAt'] >= self.from_date) & (
            dataframe_orderseries['createdAt'] < self.to_date)]
        
        orderseries = dataframe_orderseries.copy()
        # 대량 주문건 처리
        ind = orderseries['productId'].apply(lambda x : len(str(x).split('_')) > 1)
        orderseries.loc[ind,'initSubtotal'] = \
        orderseries.loc[ind,'initTotalKRW']
        orderseries.loc[ind, 'initTotalKRW'] = \
        orderseries.loc[ind]['initSubtotal'].fillna(0) + \
        orderseries.loc[ind]['initTax'].fillna(0) + \
        orderseries.loc[ind]['initHandlingFee'].fillna(0)
        orderseries = orderseries.drop(['initTax'], axis = 1)
        
        orderseries['isBulkOrder'] = \
        orderseries['productId'].apply(lambda x : len(str(x).split('_')) > 1)

        orderseries.loc[orderseries['isBulkOrder'], 'finalSubtotal'] = \
        orderseries.loc[orderseries['isBulkOrder'], 'finalTotalKRW']
        # 데이터 필터링
        orderseries = orderseries[orderseries['initTotalKRW'] != 0].reset_index(drop = True)
        # 미송 후 품절 처리 내역 필터링
        orderseries_refund = \
        orderseries[(orderseries['settledPrice'] > 0) & (
                     orderseries['status_order_prod'] == 400) & (
                     orderseries['settledQuantity'] > 0) & (
                     orderseries['status_order'].apply(lambda x : x not in [0, -1])) & (
                     orderseries['waitShipmentType'] == 100)]
        
        orderseries_refund = orderseries_refund.reset_index(drop = True)
        orderseries_refund = orderseries_refund[
            orderseries_refund['initQuantity'] != orderseries_refund['finalQuantity']]
        orderseries_refund = orderseries_refund[
            orderseries_refund['finalSubtotal'] != orderseries_refund['settledPrice']]
        # 입금 완료된 내역 제외
        dm_account = \
            pd.merge(orderseries_refund,
                     Series(list(set(orderseries_refund['orderProductId']).difference(
                                         dataframe_deposit['orderProductId'].unique())),
                            name = 'orderProductId'),
                     on = 'orderProductId')
        
        dm_account = dm_account[['buyerId',
                                'processedDate',
                                'orderProductId',
                                'initQuantity',
                                'finalQuantity',
                                'settledQuantity', 
                                'initKRW',
                                'finalSubtotal',
                                'settledPrice',
                                'finalKRW',
                                'initTotalKRW',
                                'finalTotalKRW']]
        self.dm_account = dm_account.copy()

In [28]:
omittedrefund = ETLOmittedRefund('restore.cy1gcw3mvrbf.ap-northeast-2.rds.amazonaws.com',
                              'linkshops',
                              '5432',
                              'linkshops',
                              '$dpdlvmflf4',
                              'ji.kwon@linkshops.com','cjswp25*',
                              '2022-11-01', '2022-12-29',
                                client)

In [29]:
omittedrefund.run()

Table : deposit Load Complete
NoSQLTable : dm_orderseries Load Complete
NoSQLTable : dm_seller Load Complete


In [30]:
omittedrefund.dm_account.to_excel('회계_환불생성누락_20221229.xlsx', index = False)

## 셀러 유출입 통계

In [14]:
class ETLSellerBounceStat(DataExtractor):
    def __init__(self, host_ip, DBname, port, username, password, email_id, email_pw, from_date, to_date, client):
        super().__init__(host_ip, DBname, port, username, password, email_id, email_pw, client)
        self.from_date = from_date
        self.to_date = to_date

    def load_table(self, demo_test=False):
        self.seller_data = {'SQL': "SELECT \"id\"," + (
                             "\"roles\",") + (
                             "\"data\"") + (
                             " FROM users"),
                            'columns': ['sellerId',
                                        'roles',
                                        'data']}
        
        self.history = {'SQL': "SELECT \"target_id\"," + (
                           "\"key\",") + (
                           "\"before\",") + (
                           "\"after\",") + (
                           "\"type\",") + (
                           "\"updated_at\" FROM tb_update_history ") + (
                           "where \"key\"='status'"),
                        'columns': ['sellerId',
                                    'key',
                                    'before',
                                    'after',
                                    'type',
                                    'historyUpdatedAt']}
        
        self.NOSQL = {'dm_seller': {'SQL': {}}}
        
        super().load_table()

    def run(self, threshold_membership = 1):
        def type_extractor(x, types, recursive=None):
            try:
                if recursive is None:
                    return x[0][types]
                else:
                    return type_extractor([x[0][types]],
                                          types=recursive)
            except:
                return 'NaN'
            
        def making_membership_history_dictionary(x):
            dictionary = dict()
            keys = str(int(x['before'])) + ' -> ' + str(int(x['after']))
            history_dict = pd.concat([x[['historyUpdatedAt', 'historyUpdatedAtShift']].astype('str'),
                                      Series(x['historyUpdatedAtDeriv'].days,
                                             index=['historyUpdatedAtDeriv'])]).to_dict()
            if (isinstance(history_dict, float) != True):
                dictionary[keys] = history_dict
                return dictionary
            
        def history_date_shifter(x):
            col_where = np.where(x.columns == 'historyUpdatedAt')[0]
            x = x.reset_index(drop=True)
            x = x.sort_values('historyUpdatedAt')
            if x.shape[0] > 1:
                x['historyUpdatedAtShift'] = x.iloc[1:, col_where[0]].reset_index(drop=True)
            else:
                x['historyUpdatedAtShift'] = np.nan
            return x    
        
        if not hasattr(self, 'dataframe_dm_seller'):
            self.load_table()
        
        dataframe_seller_data = self.dataframe_seller_data.copy()
        dataframe_seller = self.dataframe_dm_seller.copy()
        dataframe_update_history = self.dataframe_history.copy()
        
        dataframe_seller_data['platform'] = \
            dataframe_seller_data['data'].apply(lambda x : type_extractor([x], 'platform'))
        
        dataframe_seller = \
            pd.merge(dataframe_seller,
                     dataframe_seller_data[['sellerId','platform']],
                     on = 'sellerId',
                    how = 'left')
        dataframe_seller = \
        dataframe_seller[~dataframe_seller['status_brand'].isna()].reset_index(drop = True)
        dataframe_seller = \
        dataframe_seller[~((dataframe_seller['createdAt'].isna()) & (~dataframe_seller['sellerId'].isna()))]
        dataframe_only_seller = dataframe_seller[['sellerId',
                                                  'status_seller',
                                                  'createdAt',
                                                  'updatedAt',
                                                  'platform']].drop_duplicates().reset_index(drop = True)
        time_table = \
                    pd.date_range(dataframe_only_seller['createdAt'].min().date(),
                                  datetime.datetime.now().date(),
                                  freq = 'd')
        time_table = DataFrame({'index' : []}, columns = time_table)
        time_table_trans = time_table.T
        time_table_trans.index.name = 'historyUpdatedAt'
        
        dataframe_history = \
        dataframe_update_history[dataframe_update_history['type'] == 1].sort_values('historyUpdatedAt')
        dataframe_history = pd.merge(dataframe_only_seller, 
                             dataframe_history, 
                             on=['sellerId'])
        dataframe_history = dataframe_history.sort_values(['sellerId','historyUpdatedAt'])
        temp_1 = pd.merge(dataframe_only_seller,
                          Series(list(set(dataframe_only_seller['sellerId'].unique()).difference(
                                    dataframe_history['sellerId'].unique())),
                                 name = 'sellerId'),
                         on = 'sellerId').dropna(subset = 'createdAt').sort_values('createdAt')
        temp_1 = temp_1.reset_index(drop = True)
        temp_1_100 = temp_1[temp_1['status_seller'] == 100]
        temp_1_100_l3 = temp_1_100[temp_1_100['platform'] == 'NaN']
        temp_1_others = temp_1[temp_1['status_seller'] != 100]
        temp_1_100_others = temp_1_100[temp_1_100['platform'] != 'NaN']
        
        time_table_result = list()
        time_table_iter_temp = DataFrame()

        start_time = time.time()
        time_now = datetime.datetime.now().date()
        #### 현재 status 100 중 l3에서 생성된것으로 추정되는 사례 처리
        for num, i in temp_1_100_l3.iterrows():
            time_table_temp = \
            pd.date_range(i['createdAt'].date(),
                          time_now,
                          freq = 'd')
            time_table_temp = DataFrame({'columns' : time_table_temp})

            time_table_temp[i['sellerId']] = i['status_seller']
            time_table_temp = time_table_temp.T
            time_table_temp.columns = time_table_temp.loc['columns',:]
            time_table_temp = time_table_temp.drop(['columns'], axis = 0)
            time_table_temp.columns.name = None
            time_table_temp = \
            pd.concat([time_table_temp,
                     time_table], axis = 0)
            previous_create = \
            time_table_temp.loc[:,time_table_temp.columns[time_table_temp.columns <= i['createdAt']]].fillna(-9999)
            after_create = \
            time_table_temp.loc[:,time_table_temp.columns[time_table_temp.columns > i['createdAt']]].fillna(i['status_seller'])
            time_table_temp = pd.concat([previous_create, after_create], axis = 1)
            time_table_result.append(time_table_temp)
            if (num % 500) == 0:
                print(time.time() - start_time)
                print(num)
        time_table_result = pd.concat(time_table_result)          
        time_table_result_100_others = list()
        time_table_iter_temp = DataFrame()
        #### status 100 중 L3 이외에 경우에서 생성된 것으로 추정되는 사례
        start_time = time.time()
        time_now = datetime.datetime.now().date()
        for num, i in temp_1_100_others.iterrows():
            time_table_temp = \
            pd.date_range(i['createdAt'].date(),
                          time_now,
                          freq = 'd')
            time_table_temp = DataFrame({'columns' : time_table_temp})

            time_table_temp[i['sellerId']] = i['status_seller']
            time_table_temp = time_table_temp.T
            time_table_temp.columns = time_table_temp.loc['columns',:]
            time_table_temp = time_table_temp.drop(['columns'], axis = 0)
            time_table_temp.columns.name = None
            time_table_temp = \
            pd.concat([time_table_temp,
                     time_table], axis = 0)

            previous_create = \
            time_table_temp.loc[:,time_table_temp.columns[time_table_temp.columns <= i['createdAt']]].fillna(-9999)
            after_create = \
            time_table_temp.loc[:,time_table_temp.columns[time_table_temp.columns > i['createdAt']]].fillna(i['status_seller'])
            time_table_temp = pd.concat([previous_create, after_create], axis = 1)
            time_table_result_100_others.append(time_table_temp)
            if (num % 500) == 0:
                print(time.time() - start_time)
                print(num)   
        time_table_result_100_others = pd.concat(time_table_result_100_others)
        #### status 100 이외의 사례들 처리
        time_table_result_others = list()
        time_table_iter_temp = DataFrame()
        start_time = time.time()
        time_now = datetime.datetime.now().date()
        for num, i in temp_1_others.iterrows():
            time_table_temp = \
            pd.date_range(i['updatedAt'].date(),
                          time_now,
                          freq = 'd')
            time_table_temp = DataFrame({'columns' : time_table_temp})

            time_table_temp[i['sellerId']] = i['status_seller']
            time_table_temp = time_table_temp.T
            time_table_temp.columns = time_table_temp.loc['columns',:]
            time_table_temp = time_table_temp.drop(['columns'], axis = 0)
            time_table_temp.columns.name = None
            time_table_temp = \
            pd.concat([time_table_temp,
                     time_table], axis = 0)
            previous_create = \
            time_table_temp.loc[:,time_table_temp.columns[time_table_temp.columns <= i['createdAt']]].fillna(-9999)
            after_create = \
            time_table_temp.loc[:,time_table_temp.columns[time_table_temp.columns > i['createdAt']]].fillna(i['status_seller'])
            time_table_temp = pd.concat([previous_create, after_create], axis = 1)
            time_table_result_others.append(time_table_temp)
            if (num % 500) == 0:
                print(time.time() - start_time)
                print(num)
        time_table_result_others = pd.concat(time_table_result_others)
        time_table_result = pd.concat([time_table_result,
                              time_table_result_others,
                              time_table_result_100_others])
        #### 히스토리가 존재하는 사례들 처리
        dm_seller_for_membership = dataframe_history[~dataframe_history[
                'key'].isna()].sort_values('historyUpdatedAt').reset_index()
        col_where = np.where(dm_seller_for_membership.columns == 'historyUpdatedAt')[0]
        dm_seller_for_membership = dm_seller_for_membership.groupby('sellerId').apply(
            lambda x: history_date_shifter(x)).reset_index(drop=True)
        dm_seller_for_membership['historyUpdatedAtShift'] = (
            dm_seller_for_membership['historyUpdatedAtShift'].fillna(pd._libs.NaT))
        dm_seller_for_membership['historyUpdatedAt'] = (
            dm_seller_for_membership['historyUpdatedAt'].fillna(pd._libs.NaT))
        dm_seller_for_membership['historyUpdatedAtDeriv'] = dm_seller_for_membership.apply(
                lambda x: x['historyUpdatedAtShift'] - x['historyUpdatedAt'], axis=1)
        dm_seller_for_membership.loc[dm_seller_for_membership['historyUpdatedAtShift'].isna(),
                             'historyUpdatedAtShift'] = (
        dm_seller_for_membership.loc[dm_seller_for_membership['historyUpdatedAtShift'].isna(),
                                 'historyUpdatedAtShift'].fillna('current'))
        dm_seller_for_membership['activeYn'] = 'Y'
        dm_seller_for_membership.set_index('index', inplace=True)
        dm_seller_for_membership.index.name = ''
        brandId = dm_seller_for_membership['sellerId'].unique()
        dm_seller_for_membership = dm_seller_for_membership.dropna(subset = ['after']).reset_index(drop = True)
        dm_seller_for_membership = dm_seller_for_membership.dropna(subset = ['before']).reset_index(drop = True)
        history_frame = DataFrame()
        for i in brandId:
            temp = dm_seller_for_membership[dm_seller_for_membership['sellerId'] == i]
            temp = temp[~temp['historyUpdatedAtShift'].isna()]
            if temp.shape[0] > 0:
                membership_history = list(temp.apply(
                    lambda x: making_membership_history_dictionary(x), axis=1))

                membership_history = [i for i in membership_history if i is not None]
                before_membership_date = [
                    list(i.values())[0]['historyUpdatedAt'] for i in membership_history]
                if len(before_membership_date) > 1:
                    before_membership_date = before_membership_date[len(before_membership_date) - 2]
                elif len(before_membership_date) == 1:
                    before_membership_date = before_membership_date
                else:
                    before_membership_date = np.nan
                before_membership_due = [
                    list(i.values())[0]['historyUpdatedAtShift'] for i in membership_history]
                if len(before_membership_due) > 1:
                    before_membership_due = before_membership_due[len(before_membership_due) - 2]
                elif len(before_membership_due) == 1:
                    before_membership_due = before_membership_due
                else:
                    before_membership_due = np.nan
                membership_term = [
                    list(i.values())[0]['historyUpdatedAtDeriv'] for i in membership_history]
                if len(membership_term) > 1:
                    membership_term = membership_term[len(membership_term) - 2]
                elif len(membership_term) == 1:
                    membership_term = membership_term
                else:
                    membership_term = np.nan
                if isinstance(membership_term, pd._libs.NaTType):
                    print(membership_term)
                temp = DataFrame({'sellerId': i,
                                  'membershipHistory': [membership_history],
                                  'before_membership_date': before_membership_date,
                                  'before_membership_due': before_membership_due,
                                  'membership_term': membership_term
                                  })
                history_frame = pd.concat([history_frame, temp])
        self.history_frame = history_frame.copy()
        keys = history_frame['membershipHistory'].apply(
                            lambda x : DataFrame([Series(i.keys()).str.split('->')[0] for i in x]))
        keys = pd.concat(keys.values).reset_index(drop = True)
        keys = keys.astype('int')
        history_seller_frame = pd.merge(dataframe_only_seller[['sellerId','status_seller']],
                                         history_frame,
                                         on = 'sellerId')
        time_table_result_2 = list()
        for sellerId, i, status in zip(history_seller_frame['sellerId'], 
                               history_seller_frame['membershipHistory'],
                               history_seller_frame['status_seller']):
            temp = pd.concat([DataFrame(j).T for j in i])
            temp.index = [int(Series(j.keys()).str.split('->')[0][-1]) for j in i]
            temp['historyUpdatedAt'] = temp['historyUpdatedAt'].apply(
                lambda x : pd.Timestamp(x)).dt.tz_convert('Asia/Seoul').dt.date
            temp.loc[temp['historyUpdatedAtShift'] == 'current', 
                     'historyUpdatedAtShift'] = pd.Timestamp(datetime.datetime.now(), tz = 'UTC')

            temp['historyUpdatedAtShift'] = \
            temp['historyUpdatedAtShift'].apply(lambda x : pd.Timestamp(x)).dt.tz_convert('Asia/Seoul').dt.date
            temp = temp.groupby('historyUpdatedAt').apply(lambda x: x.iloc[-1].name)

            if temp.iloc[-1] != status:
                temp.iloc[-1] = status

            time_table_temp = \
            pd.merge(time_table_trans, DataFrame(temp), 
                     right_index = True, left_index = True, how = 'outer').fillna(method = 'ffill')
            time_table_temp = time_table_temp.T.fillna(-9999)
            time_table_temp.index = [sellerId]
            time_table_result_2.append(time_table_temp)
        time_table_result_2 = pd.concat(time_table_result_2)
        seller_subscribe = pd.concat([time_table_result,
                                       time_table_result_2], axis = 0)
        seller_subscribe_summarize = seller_subscribe.apply(lambda x : x.value_counts()).fillna(0)
        seller_subscribe_summarize = seller_subscribe_summarize.drop([-9999])
        seller_subscribe_summarize.index = ['활동가능',
                                            '가입승인 대기', 
                                            '가입승인 거부', 
                                            '활동정지',
                                            '회원탈퇴',
                                            '가입승인보류']
        seller_subscribe = seller_subscribe.loc[:,seller_subscribe.columns > self.from_date]
        seller_subscribe_summarize = seller_subscribe_summarize.loc[:,seller_subscribe_summarize.columns > self.from_date]
        seller_subscribe_summarize.columns = [str(i.date()) for i in seller_subscribe_summarize.columns]
        self.seller_subscribe = seller_subscribe.copy()
        self.seller_subscribe_summarize = seller_subscribe_summarize.copy()

In [15]:
sellerbouncestat = ETLSellerBounceStat('restore.cy1gcw3mvrbf.ap-northeast-2.rds.amazonaws.com',
                              'linkshops',
                              '5432',
                              'linkshops',
                              '$dpdlvmflf4',
                              'ji.kwon@linkshops.com','cjswp25*',
                              '2020-01-01', '2023-01-06',
                                client)

In [16]:
sellerbouncestat.run()

Table : history Load Complete
Table : seller_data Load Complete
NoSQLTable : dm_seller Load Complete
0.06841301918029785
0
44.31620454788208
500
84.06468439102173
1000
121.58526754379272
1500
156.46416187286377
2000
182.63468098640442
2500
204.07898664474487
3000
228.89697408676147
4000
241.5376753807068
4500
275.657799243927
5000
310.25734782218933
5500
341.0278079509735
6000
48.95312023162842
3500


In [19]:
sellerbouncestat.seller_subscribe_summarize

2020-01-02  2020-01-03  2020-01-04  2020-01-05  2020-01-06  \
활동가능           6166        6168        6168        6168        6172   
가입승인 대기           1           1           1           1           1   
가입승인 거부          87          87          87          87          87   
활동정지             93          93          93          93          93   
회원탈퇴           2397        2399        2399        2399        2399   
가입승인보류          121         122         122         122         121   

         2020-01-07  2020-01-08  2020-01-09  2020-01-10  2020-01-11  ...  \
활동가능           6175        6181      6183.0      6191.0      6191.0  ...   
가입승인 대기           1           1         0.0         0.0         0.0  ...   
가입승인 거부          87          87        87.0        87.0        87.0  ...   
활동정지             93          93        93.0        94.0        94.0  ...   
회원탈퇴           2400        2400      2401.0      2402.0      2402.0  ...   
가입승인보류          121         121       121.0       121.0       121.0  ...   

         2022-12-28  2022-12-29  2022-12-30  2022-12-31  2023-01-01  \
활동가능           8984        8984        8986        8986        8986   
가입승인 대기          73          73          74          74          74   
가입승인 거부         184         184         184         184         184   
활동정지            752         752         752         752         752   
회원탈퇴           2726        2726        2726        2726        2726   
가입승인보류          246         246         246         246         246   

         2023-01-02  2023-01-03  2023-01-04  2023-01-05  2023-01-06  
활동가능           8989        8992        8994        8998      9000.0  
가입승인 대기          74          75          76          77        77.0  
가입승인 거부         184         184         184         184       184.0  
활동정지            752         752         752         752       752.0  
회원탈퇴           2726        2726        2726        2726      2726.0  
가입승인보류          246         247         247         247       247.0  

[6 rows x 1101 columns]